In [13]:
#Importamos la librería pandas que necesitamos para la lectura, conversión y limpieza de los datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np
import re 

#Importamos librerías necesarias para la visualización
# -----------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames
pd.set_option('display.max_rows', None) # para poder visualizar todas las filas de los DataFrames

In [7]:
# Importar librería para la conexión con MySQL
# -----------------------------------------------------------------------
import mysql.connector
from mysql.connector import errorcode



In [4]:
df_HR = pd.read_csv('df_HR_limpio.csv')
df_HR.head()

,Unnamed: 0,Age,Attrition,BusinessTravel,DailyRate,DistanceFromHome,Education,EducationField,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,DateBirth,RemoteWork
0,0,51,No,Unknown,684.0,6,3,Unknown,"162,0",1,M,51.0,3,5,Research Director,3,Unknown,19537.0,6462,7,No,13,"3,0",3,0,Unknown,5,3.000000,20,15,15,1972,Yes
1,1,52,No,Unknown,699.0,1,4,Life Sciences,"259,0",3,M,65.0,2,5,Manager,3,Unknown,19999.0,5678,0,Unknown,14,"3,0",1,1,"34,0",5,3.000000,33,11,9,1971,1
2,2,42,No,travel_rarely,532.0,4,2,Technical Degree,"319,0",3,M,58.0,3,5,Manager,4,Married,19232.0,4933,1,No,11,"3,0",4,0,"22,0",3,2.766568,22,11,15,1981,1
3,3,47,No,travel_rarely,359.0,2,4,Medical,Unknown,1,F,82.0,3,4,Research Director,3,Married,17169.0,26703,3,Unknown,19,"3,0",2,2,Unknown,2,2.766568,20,5,6,1976,False
4,5,48,No,Unknown,117.0,22,3,Medical,"1900,0",4,F,58.0,3,4,Manager,4,Unknown,17174.0,2437,3,No,11,"3,0",2,1,Unknown,3,3.000000,22,4,7,1975,Yes


In [5]:
df_Employees = df_HR[["Age", "Gender", "MaritalStatus", "DateBirth", "Education", "EducationField", "Attrition", "BusinessTravel", "DistanceFromHome"]]

df_JobDetails = df_HR[["JobRole", "JobLevel", "NumCompaniesWorked", "TotalWorkingYears", "YearsAtCompany", "YearsSinceLastPromotion", "YearsWithCurrManager"]]

df_Compensation = df_HR[["DailyRate", "HourlyRate", "MonthlyIncome", "MonthlyRate", "StockOptionLevel", "PercentSalaryHike", "OverTime", "TrainingTimesLastYear", "RemoteWork"]]

df_Satisfaction = df_HR[["EnvironmentSatisfaction", "JobInvolvement", "JobSatisfaction", "RelationshipSatisfaction", "WorkLifeBalance", "PerformanceRating"]]

df_employees

In [14]:
lista_de_tuplas = [tuple(x) for x in df_Employees.to_records(index=False)] 



In [15]:
lista_de_tuplas

[(51, 'M', 'Unknown', 1972, 3, 'Unknown', 'No', 'Unknown', 6),
 (52, 'M', 'Unknown', 1971, 4, 'Life Sciences', 'No', 'Unknown', 1),
 (42, 'M', 'Married', 1981, 2, 'Technical Degree', 'No', 'travel_rarely', 4),
 (47, 'F', 'Married', 1976, 4, 'Medical', 'No', 'travel_rarely', 2),
 (48, 'F', 'Unknown', 1975, 3, 'Medical', 'No', 'Unknown', 22),
 (59, 'F', 'Unknown', 1964, 3, 'Life Sciences', 'No', 'Unknown', 25),
 (42, 'M', 'Married', 1981, 1, 'Unknown', 'No', 'travel_rarely', 1),
 (41, 'M', 'Unknown', 1982, 3, 'Unknown', 'No', 'travel_frequently', 9),
 (41, 'F', 'Unknown', 1982, 1, 'Unknown', 'No', 'non-travel', 7),
 (41, 'F', 'Married', 1982, 3, 'Unknown', 'Yes', 'travel_rarely', 12),
 (40, 'F', 'Unknown', 1983, 4, 'Unknown', 'No', 'travel_rarely', 2),
 (58, 'M', 'Single', 1965, 4, 'Unknown', 'Yes', 'travel_rarely', 2),
 (45, 'M', 'Unknown', 1978, 1, 'Unknown', 'No', 'non-travel', 1),
 (33, 'F', 'Single', 1990, 2, 'Life Sciences', 'No', 'Unknown', 1),
 (47, 'F', 'Married', 1976, 2, 'Othe

In [19]:
df_Employees.shape

(1080, 9)

Crear base de datos

In [8]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1')


mycursor = cnx.cursor()
try:
    mycursor.execute("CREATE DATABASE BBDD_ABC_Corporation")
    print(mycursor)
except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

CMySQLCursor: CREATE DATABASE BBDD_ABC_Corporation


Creacion de tablas

In [9]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='BBDD_ABC_Corporation')


mycursor = cnx.cursor()
try:
    mycursor.execute("""
        CREATE TABLE Employees (
        EmployeeNumber INT PRIMARY KEY,
        Age INT,
        Gender VARCHAR(100),
        MaritalStatus VARCHAR(100),
        DateBirth DATE,
        Education INT,
        EducationField VARCHAR(100),
        Attrition VARCHAR(100),
        BusinessTravel VARCHAR(100),
        DistanceFromHome INT
    )
    """)
    print(mycursor)
except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

CMySQLCursor: 
        CREATE TABLE Employees (
      ..


Insercion de datos

In [20]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='BBDD_ABC_Corporation')


mycursor = cnx.cursor()
sql = "INSERT INTO Employees (EmployeeNumber,Age, Gender, MaritalStatus, DateBirth, Education, EducationField, Attrition, BusinessTravel, DistanceFromHome) VALUES (%s, %s)"
val = [
(51, 'M', 'Unknown', 1972, 3, 'Unknown', 'No', 'Unknown', 6),
 (52, 'M', 'Unknown', 1971, 4, 'Life Sciences', 'No', 'Unknown', 1),
 (42, 'M', 'Married', 1981, 2, 'Technical Degree', 'No', 'travel_rarely', 4),
 (47, 'F', 'Married', 1976, 4, 'Medical', 'No', 'travel_rarely', 2),
 (48, 'F', 'Unknown', 1975, 3, 'Medical', 'No', 'Unknown', 22),
 (59, 'F', 'Unknown', 1964, 3, 'Life Sciences', 'No', 'Unknown', 25),
 (42, 'M', 'Married', 1981, 1, 'Unknown', 'No', 'travel_rarely', 1),
 (41, 'M', 'Unknown', 1982, 3, 'Unknown', 'No', 'travel_frequently', 9),
 (41, 'F', 'Unknown', 1982, 1, 'Unknown', 'No', 'non-travel', 7),
 (41, 'F', 'Married', 1982, 3, 'Unknown', 'Yes', 'travel_rarely', 12),
 (40, 'F', 'Unknown', 1983, 4, 'Unknown', 'No', 'travel_rarely', 2),
 (58, 'M', 'Single', 1965, 4, 'Unknown', 'Yes', 'travel_rarely', 2),
 (45, 'M', 'Unknown', 1978, 1, 'Unknown', 'No', 'non-travel', 1),
 (33, 'F', 'Single', 1990, 2, 'Life Sciences', 'No', 'Unknown', 1),
 (47, 'F', 'Married', 1976, 2, 'Other', 'No', 'Unknown', 2),
 (36, 'F', 'Single', 1987, 4, 'Life Sciences', 'No', 'Unknown', 2),
 (34, 'M', 'Unknown', 1989, 3, 'Life Sciences', 'No', 'non-travel', 10),
 (41, 'F', 'Divorced', 1982, 1, 'Unknown', 'No', 'Unknown', 37),
 (47, 'M', 'Unknown', 1976, 3, 'Medical', 'No', 'Unknown', 25),
 (53, 'F', 'Unknown', 1970, 3, 'Unknown', 'No', 'travel_rarely', 5),
 (35, 'M', 'Single', 1988, 4, 'Marketing', 'No', 'Unknown', 1),
 (46, 'F', 'Divorced', 1977, 2, 'Unknown', 'No', 'travel_rarely', 1),
 (43, 'F', 'Married', 1980, 3, 'Unknown', 'No', 'Unknown', 16),
 (36, 'M', 'Married', 1987, 3, 'Life Sciences', 'No', 'Unknown', 6),
 (40, 'M', 'Married', 1983, 4, 'Unknown', 'No', 'travel_frequently', 5),
 (32, 'F', 'Married', 1991, 3, 'Unknown', 'No', 'travel_rarely', 4),
 (53, 'M', 'Divorced', 1970, 2, 'Medical', 'No', 'Unknown', 2),
 (36, 'F', 'Single', 1987, 4, 'Unknown', 'Yes', 'travel_rarely', 16),
 (46, 'F', 'Divorced', 1977, 4, 'Life Sciences', 'No', 'Unknown', 8),
 (38, 'M', 'Married', 1985, 4, 'Unknown', 'No', 'travel_rarely', 1),
 (51, 'F', 'Single', 1972, 4, 'Unknown', 'No', 'Unknown', 1),
 (43, 'M', 'Married', 1980, 2, 'Unknown', 'No', 'Unknown', 26),
 (37, 'F', 'Single', 1986, 3, 'Life Sciences', 'No', 'Unknown', 1),
 (32, 'F', 'Married', 1991, 2, 'Technical Degree', 'No', 'travel_rarely', 8),
 (46, 'F', 'Unknown', 1977, 3, 'Life Sciences', 'No', 'Unknown', 3),
 (43, 'F', 'Married', 1980, 3, 'Medical', 'No', 'Unknown', 6),
 (56, 'M', 'Divorced', 1967, 4, 'Life Sciences', 'No', 'non-travel', 1),
 (38, 'F', 'Unknown', 1985, 3, 'Medical', 'No', 'Unknown', 6),
 (49, 'F', 'Unknown', 1974, 4, 'Medical', 'No', 'Unknown', 26),
 (40, 'M', 'Married', 1983, 4, 'Unknown', 'No', 'Unknown', 4),
 (36, 'M', 'Single', 1987, 2, 'Unknown', 'No', 'Unknown', 5),
 (37, 'F', 'Married', 1986, 5, 'Unknown', 'No', 'Unknown', 25),
 (35, 'M', 'Unknown', 1988, 4, 'Unknown', 'No', 'Unknown', 6),
 (33, 'M', 'Divorced', 1990, 3, 'Life Sciences', 'No', 'travel_frequently', 6),
 (50, 'M', 'Married', 1973, 3, 'Unknown', 'No', 'travel_rarely', 24),
 (48, 'M', 'Single', 1975, 4, 'Unknown', 'No', 'travel_rarely', 4),
 (44, 'F', 'Married', 1979, 3, 'Unknown', 'No', 'travel_rarely', 28),
 (44, 'M', 'Single', 1979, 4, 'Unknown', 'No', 'travel_rarely', 29),
 (40, 'F', 'Unknown', 1983, 1, 'Unknown', 'No', 'Unknown', 25),
 (49, 'F', 'Unknown', 1974, 1, 'Life Sciences', 'No', 'Unknown', 6),
 (38, 'M', 'Unknown', 1985, 3, 'Life Sciences', 'No', 'travel_frequently', 1),
 (33, 'M', 'Unknown', 1990, 3, 'Unknown', 'No', 'Unknown', 21),
 (38, 'F', 'Divorced', 1985, 3, 'Unknown', 'No', 'Unknown', 8),
 (37, 'F', 'Unknown', 1986, 3, 'Life Sciences', 'No', 'travel_rarely', 10),
 (30, 'M', 'Married', 1993, 3, 'Unknown', 'No', 'Unknown', 2),
 (58, 'M', 'Single', 1965, 4, 'Unknown', 'No', 'travel_rarely', 18),
 (34, 'F', 'Single', 1989, 3, 'Unknown', 'No', 'travel_rarely', 5),
 (38, 'M', 'Divorced', 1985, 3, 'Life Sciences', 'No', 'Unknown', 3),
 (55, 'M', 'Single', 1968, 3, 'Medical', 'No', 'travel_rarely', 1),
 (31, 'F', 'Married', 1992, 2, 'Unknown', 'No', 'Unknown', 30),
 (50, 'M', 'Married', 1973, 4, 'Life Sciences', 'No', 'travel_rarely', 27),
 (35, 'M', 'Married', 1988, 3, 'Technical Degree', 'No', 'Unknown', 25),
 (35, 'F', 'Divorced', 1988, 4, 'Unknown', 'No', 'Unknown', 25),
 (45, 'F', 'Married', 1978, 4, 'Life Sciences', 'No', 'Unknown', 9),
 (33, 'M', 'Married', 1990, 3, 'Medical', 'No', 'Unknown', 9),
 (45, 'F', 'Unknown', 1978, 3, 'Marketing', 'Yes', 'Unknown', 13),
 (34, 'M', 'Unknown', 1989, 2, 'Technical Degree', 'No', 'travel_rarely', 7),
 (43, 'M', 'Unknown', 1980, 2, 'Unknown', 'No', 'travel_rarely', 15),
 (34, 'M', 'Unknown', 1989, 2, 'Medical', 'No', 'Unknown', 5),
 (38, 'M', 'Divorced', 1985, 3, 'Unknown', 'No', 'Unknown', 2),
 (34, 'M', 'Married', 1989, 4, 'Other', 'No', 'Unknown', 22),
 (32, 'F', 'Unknown', 1991, 4, 'Unknown', 'No', 'travel_rarely', 31),
 (29, 'F', 'Divorced', 1994, 4, 'Life Sciences', 'No', 'travel_rarely', 21),
 (30, 'M', 'Married', 1993, 3, 'Unknown', 'No', 'Unknown', 9),
 (45, 'M', 'Unknown', 1978, 3, 'Life Sciences', 'No', 'Unknown', 9),
 (30, 'F', 'Unknown', 1993, 3, 'Unknown', 'No', 'travel_rarely', 5),
 (30, 'M', 'Unknown', 1993, 3, 'Unknown', 'No', 'Unknown', 9),
 (34, 'M', 'Married', 1989, 1, 'Unknown', 'No', 'travel_frequently', 29),
 (34, 'M', 'Unknown', 1989, 4, 'Life Sciences', 'No', 'Unknown', 39),
 (35, 'M', 'Married', 1988, 3, 'Unknown', 'No', 'travel_rarely', 16),
 (39, 'M', 'Married', 1984, 1, 'Unknown', 'No', 'Unknown', 10),
 (39, 'M', 'Married', 1984, 2, 'Unknown', 'No', 'Unknown', 7),
 (45, 'M', 'Unknown', 1978, 4, 'Unknown', 'No', 'travel_rarely', 8),
 (30, 'F', 'Unknown', 1993, 3, 'Marketing', 'No', 'travel_rarely', 5),
 (36, 'M', 'Married', 1987, 4, 'Medical', 'No', 'Unknown', 3),
 (36, 'M', 'Married', 1987, 2, 'Unknown', 'No', 'travel_rarely', 2),
 (55, 'F', 'Single', 1968, 5, 'Unknown', 'No', 'travel_rarely', 18),
 (35, 'M', 'Unknown', 1988, 3, 'Unknown', 'No', 'Unknown', 1),
 (52, 'M', 'Divorced', 1971, 2, 'Unknown', 'No', 'travel_rarely', 6),
 (49, 'M', 'Unknown', 1974, 4, 'Life Sciences', 'No', 'Unknown', 18),
 (34, 'M', 'Married', 1989, 5, 'Unknown', 'No', 'travel_rarely', 1),
 (33, 'M', 'Single', 1990, 4, 'Other', 'Yes', 'travel_rarely', 1),
 (33, 'M', 'Married', 1990, 3, 'Medical', 'No', 'Unknown', 7),
 (53, 'F', 'Unknown', 1970, 4, 'Unknown', 'No', 'travel_rarely', 23),
 (49, 'F', 'Married', 1974, 2, 'Other', 'No', 'travel_rarely', 8),
 (35, 'M', 'Unknown', 1988, 4, 'Unknown', 'No', 'Unknown', 21),
 (38, 'F', 'Married', 1985, 1, 'Life Sciences', 'No', 'travel_rarely', 17),
 (46, 'M', 'Married', 1977, 1, 'Marketing', 'No', 'Unknown', 3),
 (42, 'M', 'Single', 1981, 3, 'Life Sciences', 'Yes', 'travel_frequently', 15),
 (34, 'M', 'Single', 1989, 3, 'Unknown', 'No', 'travel_frequently', 1),
 (31, 'F', 'Single', 1992, 3, 'Medical', 'No', 'travel_rarely', 24),
 (44, 'M', 'Married', 1979, 3, 'Unknown', 'No', 'Unknown', 5),
 (55, 'M', 'Divorced', 1968, 1, 'Unknown', 'No', 'non-travel', 8),
 (44, 'M', 'Divorced', 1979, 2, 'Life Sciences', 'No', 'travel_rarely', 4),
 (39, 'F', 'Divorced', 1984, 4, 'Unknown', 'No', 'Unknown', 13),
 (55, 'F', 'Unknown', 1968, 2, 'Medical', 'No', 'travel_rarely', 2),
 (51, 'M', 'Unknown', 1972, 3, 'Life Sciences', 'No', 'Unknown', 5),
 (58, 'F', 'Married', 1965, 4, 'Medical', 'Yes', 'Unknown', 14),
 (29, 'M', 'Single', 1994, 3, 'Unknown', 'No', 'travel_rarely', 17),
 (34, 'M', 'Single', 1989, 3, 'Other', 'No', 'Unknown', 3),
 (42, 'F', 'Unknown', 1981, 2, 'Unknown', 'No', 'travel_frequently', 9),
 (34, 'F', 'Single', 1989, 4, 'Unknown', 'Yes', 'travel_rarely', 24),
 (54, 'F', 'Unknown', 1969, 4, 'Medical', 'No', 'travel_frequently', 11),
 (34, 'M', 'Unknown', 1989, 3, 'Unknown', 'No', 'travel_rarely', 1),
 (29, 'M', 'Single', 1994, 1, 'Medical', 'No', 'Unknown', 3),
 (35, 'M', 'Unknown', 1988, 3, 'Life Sciences', 'No', 'travel_rarely', 28),
 (35, 'M', 'Unknown', 1988, 4, 'Other', 'Yes', 'Unknown', 12),
 (34, 'F', 'Married', 1989, 3, 'Life Sciences', 'No', 'travel_rarely', 19),
 (43, 'M', 'Unknown', 1980, 3, 'Unknown', 'No', 'Unknown', 2),
 (43, 'M', 'Married', 1980, 3, 'Life Sciences', 'No', 'travel_rarely', 11),
 (42, 'M', 'Single', 1981, 2, 'Unknown', 'No', 'Unknown', 23),
 (40, 'F', 'Married', 1983, 3, 'Technical Degree', 'No', 'Unknown', 11),
 (56, 'M', 'Married', 1967, 2, 'Technical Degree', 'Yes', 'Unknown', 7),
 (39, 'F', 'Unknown', 1984, 1, 'Marketing', 'No', 'travel_rarely', 33),
 (30, 'F', 'Unknown', 1993, 2, 'Unknown', 'No', 'travel_frequently', 1),
 (32, 'M', 'Married', 1991, 3, 'Medical', 'No', 'travel_rarely', 1),
 (30, 'M', 'Married', 1993, 1, 'Life Sciences', 'No', 'travel_rarely', 1),
 (46, 'M', 'Unknown', 1977, 2, 'Medical', 'Yes', 'Unknown', 9),
 (30, 'F', 'Unknown', 1993, 2, 'Unknown', 'No', 'travel_rarely', 4),
 (32, 'M', 'Single', 1991, 4, 'Unknown', 'No', 'Unknown', 6),
 (32, 'F', 'Single', 1991, 3, 'Marketing', 'No', 'Unknown', 8),
 (41, 'M', 'Divorced', 1982, 3, 'Life Sciences', 'No', 'travel_rarely', 34),
 (40, 'F', 'Unknown', 1983, 1, 'Medical', 'No', 'travel_rarely', 4),
 (45, 'M', 'Married', 1978, 2, 'Unknown', 'No', 'Unknown', 2),
 (32, 'M', 'Married', 1991, 4, 'Unknown', 'Yes', 'travel_rarely', 4),
 (30, 'M', 'Unknown', 1993, 1, 'Life Sciences', 'No', 'non-travel', 1),
 (37, 'M', 'Unknown', 1986, 3, 'Unknown', 'No', 'Unknown', 11),
 (40, 'M', 'Unknown', 1983, 2, 'Technical Degree', 'No', 'Unknown', 24),
 (32, 'M', 'Married', 1991, 3, 'Life Sciences', 'No', 'travel_rarely', 7),
 (38, 'F', 'Unknown', 1985, 2, 'Life Sciences', 'No', 'Unknown', 25),
 (30, 'F', 'Married', 1993, 2, 'Unknown', 'No', 'travel_rarely', 28),
 (39, 'M', 'Unknown', 1984, 1, 'Unknown', 'No', 'Unknown', 1),
 (60, 'F', 'Unknown', 1963, 3, 'Life Sciences', 'No', 'Unknown', 7),
 (30, 'F', 'Unknown', 1993, 4, 'Other', 'No', 'Unknown', 22),
 (31, 'M', 'Divorced', 1992, 1, 'Unknown', 'No', 'travel_rarely', 9),
 (33, 'M', 'Single', 1990, 3, 'Life Sciences', 'Yes', 'Unknown', 3),
 (50, 'M', 'Married', 1973, 1, 'Medical', 'No', 'Unknown', 28),
 (32, 'M', 'Unknown', 1991, 4, 'Life Sciences', 'No', 'Unknown', 24),
 (42, 'M', 'Unknown', 1981, 3, 'Unknown', 'No', 'Unknown', 29),
 (49, 'F', 'Married', 1974, 3, 'Marketing', 'Yes', 'Unknown', 11),
 (27, 'M', 'Divorced', 1996, 1, 'Unknown', 'No', 'Unknown', 5),
 (52, 'F', 'Married', 1971, 4, 'Unknown', 'No', 'Unknown', 26),
 (32, 'M', 'Unknown', 1991, 2, 'Life Sciences', 'No', 'Unknown', 5),
 (36, 'M', 'Unknown', 1987, 1, 'Life Sciences', 'No', 'Unknown', 14),
 (52, 'M', 'Married', 1971, 2, 'Life Sciences', 'No', 'Unknown', 1),
 (33, 'M', 'Single', 1990, 3, 'Unknown', 'No', 'non-travel', 17),
 (45, 'M', 'Unknown', 1978, 2, 'Life Sciences', 'No', 'non-travel', 23),
 (36, 'F', 'Unknown', 1987, 5, 'Unknown', 'No', 'travel_rarely', 12),
 (49, 'M', 'Unknown', 1974, 4, 'Unknown', 'No', 'Unknown', 8),
 (29, 'F', 'Married', 1994, 1, 'Unknown', 'No', 'Unknown', 6),
 (37, 'F', 'Married', 1986, 3, 'Life Sciences', 'No', 'Unknown', 1),
 (28, 'F', 'Married', 1995, 3, 'Unknown', 'No', 'non-travel', 1),
 (31, 'F', 'Unknown', 1992, 4, 'Unknown', 'No', 'non-travel', 2),
 (42, 'M', 'Unknown', 1981, 1, 'Life Sciences', 'No', 'non-travel', 2),
 (40, 'M', 'Single', 1983, 3, 'Life Sciences', 'No', 'travel_rarely', 2),
 (34, 'F', 'Single', 1989, 2, 'Medical', 'No', 'Unknown', 15),
 (58, 'F', 'Unknown', 1965, 3, 'Unknown', 'No', 'travel_rarely', 1),
 (31, 'F', 'Unknown', 1992, 5, 'Unknown', 'Yes', 'travel_frequently', 1),
 (53, 'M', 'Single', 1970, 3, 'Medical', 'No', 'Unknown', 2),
 (34, 'M', 'Unknown', 1989, 3, 'Unknown', 'No', 'Unknown', 15),
 (37, 'M', 'Married', 1986, 3, 'Unknown', 'No', 'Unknown', 13),
 (42, 'M', 'Unknown', 1981, 3, 'Unknown', 'No', 'travel_frequently', 2),
 (45, 'M', 'Divorced', 1978, 4, 'Unknown', 'No', 'Unknown', 1),
 (30, 'M', 'Married', 1993, 3, 'Unknown', 'Yes', 'travel_rarely', 1),
 (38, 'M', 'Married', 1985, 3, 'Life Sciences', 'No', 'travel_rarely', 14),
 (50, 'M', 'Unknown', 1973, 5, 'Medical', 'No', 'Unknown', 20),
 (34, 'F', 'Unknown', 1989, 3, 'Unknown', 'No', 'travel_rarely', 3),
 (54, 'F', 'Unknown', 1969, 5, 'Unknown', 'No', 'travel_rarely', 22),
 (39, 'M', 'Married', 1984, 3, 'Life Sciences', 'No', 'Unknown', 1),
 (36, 'M', 'Unknown', 1987, 3, 'Life Sciences', 'No', 'Unknown', 10),
 (34, 'M', 'Unknown', 1989, 2, 'Unknown', 'No', 'travel_rarely', 8),
 (46, 'M', 'Divorced', 1977, 2, 'Life Sciences', 'No', 'travel_rarely', 2),
 (27, 'F', 'Married', 1996, 1, 'Medical', 'No', 'travel_rarely', 23),
 (31, 'M', 'Divorced', 1992, 3, 'Life Sciences', 'No', 'Unknown', 36),
 (45, 'F', 'Unknown', 1978, 2, 'Life Sciences', 'No', 'travel_rarely', 25),
 (34, 'M', 'Unknown', 1989, 3, 'Marketing', 'No', 'travel_rarely', 1),
 (35, 'F', 'Unknown', 1988, 4, 'Unknown', 'No', 'Unknown', 7),
 (39, 'M', 'Single', 1984, 4, 'Marketing', 'No', 'travel_rarely', 15),
 (40, 'F', 'Married', 1983, 2, 'Marketing', 'No', 'travel_rarely', 2),
 (47, 'F', 'Married', 1976, 1, 'Unknown', 'No', 'Unknown', 1),
 (49, 'M', 'Married', 1974, 1, 'Unknown', 'No', 'travel_frequently', 8),
 (33, 'F', 'Married', 1990, 4, 'Life Sciences', 'No', 'travel_frequently', 19),
 (42, 'M', 'Unknown', 1981, 4, 'Unknown', 'No', 'Unknown', 8),
 (31, 'M', 'Divorced', 1992, 4, 'Life Sciences', 'No', 'Unknown', 7),
 (31, 'F', 'Single', 1992, 4, 'Unknown', 'No', 'travel_rarely', 8),
 (51, 'M', 'Unknown', 1972, 4, 'Marketing', 'No', 'Unknown', 21),
 (34, 'M', 'Unknown', 1989, 3, 'Unknown', 'Yes', 'travel_frequently', 7),
 (36, 'F', 'Unknown', 1987, 2, 'Technical Degree', 'No', 'Unknown', 11),
 (30, 'M', 'Divorced', 1993, 3, 'Unknown', 'No', 'travel_frequently', 23),
 (33, 'M', 'Married', 1990, 2, 'Unknown', 'No', 'non-travel', 12),
 (31, 'M', 'Married', 1992, 4, 'Life Sciences', 'Yes', 'Unknown', 6),
 (41, 'M', 'Divorced', 1982, 1, 'Unknown', 'No', 'travel_rarely', 23),
 (26, 'F', 'Unknown', 1997, 1, 'Unknown', 'No', 'travel_rarely', 25),
 (32, 'M', 'Divorced', 1991, 2, 'Unknown', 'No', 'Unknown', 9),
 (40, 'F', 'Divorced', 1983, 3, 'Unknown', 'No', 'travel_frequently', 26),
 (53, 'M', 'Unknown', 1970, 2, 'Marketing', 'No', 'travel_rarely', 6),
 (33, 'F', 'Unknown', 1990, 3, 'Unknown', 'No', 'travel_rarely', 14),
 (51, 'F', 'Single', 1972, 3, 'Unknown', 'No', 'travel_rarely', 3),
 (27, 'F', 'Single', 1996, 3, 'Medical', 'No', 'travel_rarely', 9),
 (37, 'M', 'Unknown', 1986, 3, 'Life Sciences', 'No', 'Unknown', 21),
 (49, 'M', 'Divorced', 1974, 4, 'Unknown', 'No', 'non-travel', 18),
 (27, 'F', 'Single', 1996, 3, 'Life Sciences', 'No', 'Unknown', 20),
 (45, 'F', 'Single', 1978, 3, 'Medical', 'No', 'Unknown', 6),
 (35, 'F', 'Single', 1988, 3, 'Unknown', 'No', 'Unknown', 1),
 (52, 'M', 'Unknown', 1971, 4, 'Life Sciences', 'No', 'Unknown', 2),
 (38, 'M', 'Unknown', 1985, 2, 'Life Sciences', 'No', 'non-travel', 2),
 (46, 'F', 'Married', 1977, 3, 'Unknown', 'No', 'Unknown', 27),
 (47, 'M', 'Married', 1976, 4, 'Life Sciences', 'Yes', 'non-travel', 10),
 (35, 'F', 'Divorced', 1988, 3, 'Life Sciences', 'No', 'Unknown', 22),
 (29, 'M', 'Single', 1994, 2, 'Unknown', 'No', 'travel_frequently', 2),
 (55, 'M', 'Married', 1968, 2, 'Technical Degree', 'No', 'travel_rarely', 20),
 (26, 'M', 'Single', 1997, 4, 'Unknown', 'Yes', 'travel_rarely', 45),
 (31, 'M', 'Unknown', 1992, 3, 'Technical Degree', 'No', 'travel_rarely', 7),
 (43, 'F', 'Unknown', 1980, 4, 'Life Sciences', 'No', 'travel_frequently', 10),
 (31, 'M', 'Unknown', 1992, 3, 'Marketing', 'No', 'travel_rarely', 13),
 (33, 'M', 'Unknown', 1990, 3, 'Unknown', 'No', 'Unknown', 10),
 (43,
  'F',
  'Divorced',
  1980,
  4,
  'Life Sciences',
  'No',
  'travel_frequently',
  10),
 (29, 'M', 'Unknown', 1994, 4, 'Unknown', 'Yes', 'travel_rarely', 8),
 (27, 'F', 'Unknown', 1996, 4, 'Unknown', 'No', 'Unknown', 8),
 (26, 'M', 'Unknown', 1997, 3, 'Unknown', 'No', 'travel_rarely', 1),
 (40, 'F', 'Married', 1983, 4, 'Life Sciences', 'No', 'travel_rarely', 10),
 (41, 'M', 'Unknown', 1982, 3, 'Unknown', 'Yes', 'Unknown', 4),
 (51, 'F', 'Unknown', 1972, 2, 'Unknown', 'No', 'travel_rarely', 14),
 (41, 'M', 'Married', 1982, 3, 'Marketing', 'No', 'travel_rarely', 1),
 (35, 'M', 'Unknown', 1988, 3, 'Life Sciences', 'No', 'travel_rarely', 16),
 (48, 'F', 'Married', 1975, 5, 'Unknown', 'No', 'travel_rarely', 2),
 (26,
  'F',
  'Unknown',
  1997,
  4,
  'Life Sciences',
  'Yes',
  'travel_frequently',
  17),
 (41, 'F', 'Single', 1982, 4, 'Other', 'No', 'Unknown', 2),
 (34, 'M', 'Single', 1989, 4, 'Unknown', 'No', 'Unknown', 21),
 (31, 'M', 'Divorced', 1992, 4, 'Medical', 'No', 'travel_rarely', 6),
 (43, 'M', 'Divorced', 1980, 5, 'Unknown', 'No', 'travel_frequently', 9),
 (53, 'F', 'Single', 1970, 1, 'Unknown', 'No', 'Unknown', 1),
 (29, 'M', 'Divorced', 1994, 4, 'Life Sciences', 'No', 'Unknown', 1),
 (34, 'F', 'Unknown', 1989, 3, 'Medical', 'No', 'Unknown', 9),
 (39, 'F', 'Single', 1984, 4, 'Technical Degree', 'No', 'Unknown', 2),
 (32, 'M', 'Unknown', 1991, 3, 'Marketing', 'No', 'Unknown', 10),
 (38, 'F', 'Unknown', 1985, 4, 'Life Sciences', 'No', 'travel_rarely', 1),
 (27, 'F', 'Married', 1996, 2, 'Life Sciences', 'No', 'travel_rarely', 1),
 (47, 'M', 'Single', 1976, 4, 'Marketing', 'No', 'Unknown', 35),
 (27, 'M', 'Unknown', 1996, 2, 'Unknown', 'No', 'travel_frequently', 10),
 (41, 'M', 'Unknown', 1982, 3, 'Unknown', 'No', 'Unknown', 1),
 (36, 'F', 'Married', 1987, 3, 'Unknown', 'No', 'Unknown', 8),
 (30, 'M', 'Divorced', 1993, 5, 'Marketing', 'No', 'Unknown', 27),
 (56, 'M', 'Married', 1967, 3, 'Unknown', 'No', 'travel_rarely', 9),
 (45, 'M', 'Single', 1978, 2, 'Unknown', 'No', 'travel_rarely', 2),
 (36, 'M', 'Single', 1987, 4, 'Unknown', 'No', 'non-travel', 8),
 (29, 'F', 'Married', 1994, 3, 'Medical', 'No', 'Unknown', 27),
 (38, 'F', 'Single', 1985, 4, 'Life Sciences', 'No', 'Unknown', 2),
 (42, 'M', 'Unknown', 1981, 3, 'Unknown', 'No', 'travel_frequently', 8),
 (36, 'F', 'Unknown', 1987, 3, 'Other', 'No', 'travel_rarely', 1),
 (55, 'M', 'Single', 1968, 1, 'Medical', 'Yes', 'Unknown', 2),
 (44, 'M', 'Unknown', 1979, 2, 'Unknown', 'No', 'Unknown', 9),
 (41, 'F', 'Divorced', 1982, 4, 'Life Sciences', 'Yes', 'Unknown', 2),
 (31, 'M', 'Unknown', 1992, 4, 'Unknown', 'No', 'travel_rarely', 7),
 (26, 'M', 'Unknown', 1997, 3, 'Medical', 'No', 'travel_frequently', 28),
 (32, 'F', 'Married', 1991, 4, 'Medical', 'No', 'Unknown', 3),
 (36, 'M', 'Married', 1987, 1, 'Life Sciences', 'Yes', 'travel_rarely', 3),
 (45, 'M', 'Unknown', 1978, 2, 'Unknown', 'No', 'non-travel', 47),
 (36, 'M', 'Divorced', 1987, 4, 'Technical Degree', 'No', 'non-travel', 8),
 (54, 'F', 'Unknown', 1969, 3, 'Technical Degree', 'No', 'travel_rarely', 17),
 (30, 'F', 'Unknown', 1993, 2, 'Unknown', 'No', 'travel_rarely', 8),
 (26, 'M', 'Married', 1997, 2, 'Medical', 'No', 'Unknown', 2),
 (55, 'M', 'Married', 1968, 4, 'Medical', 'No', 'travel_frequently', 18),
 (32, 'F', 'Single', 1991, 4, 'Unknown', 'No', 'non-travel', 29),
 (53, 'F', 'Unknown', 1970, 3, 'Marketing', 'No', 'travel_frequently', 2),
 (55, 'M', 'Single', 1968, 1, 'Unknown', 'No', 'Unknown', 2),
 (28, 'M', 'Single', 1995, 2, 'Unknown', 'Yes', 'non-travel', 24),
 (28, 'F', 'Single', 1995, 3, 'Other', 'No', 'travel_rarely', 10),
 (35, 'M', 'Unknown', 1988, 3, 'Technical Degree', 'Yes', 'Unknown', 4),
 (33, 'F', 'Single', 1990, 4, 'Medical', 'Yes', 'travel_frequently', 36),
 (25, 'M', 'Married', 1998, 1, 'Technical Degree', 'Yes', 'Unknown', 4),
 (38, 'M', 'Unknown', 1985, 4, 'Unknown', 'No', 'Unknown', 10),
 (43, 'F', 'Married', 1980, 3, 'Life Sciences', 'No', 'travel_frequently', 27),
 (40, 'M', 'Unknown', 1983, 3, 'Unknown', 'No', 'travel_rarely', 14),
 (26, 'M', 'Unknown', 1997, 4, 'Unknown', 'No', 'travel_rarely', 3),
 (35, 'M', 'Divorced', 1988, 3, 'Unknown', 'No', 'travel_rarely', 2),
 (53, 'F', 'Divorced', 1970, 2, 'Medical', 'No', 'Unknown', 7),
 (35, 'M', 'Unknown', 1988, 4, 'Life Sciences', 'No', 'Unknown', 17),
 (33, 'F', 'Unknown', 1990, 3, 'Medical', 'No', 'Unknown', 25),
 (30, 'M', 'Divorced', 1993, 4, 'Life Sciences', 'No', 'travel_rarely', 7),
 (32, 'F', 'Single', 1991, 2, 'Life Sciences', 'No', 'Unknown', 2),
 (36, 'F', 'Divorced', 1987, 3, 'Technical Degree', 'No', 'Unknown', 1),
 (43, 'M', 'Unknown', 1980, 3, 'Life Sciences', 'No', 'Unknown', 3),
 (29, 'F', 'Divorced', 1994, 3, 'Unknown', 'No', 'travel_rarely', 1),
 (59, 'M', 'Single', 1964, 4, 'Unknown', 'No', 'Unknown', 24),
 (43, 'F', 'Single', 1980, 4, 'Marketing', 'No', 'travel_rarely', 4),
 (44, 'F', 'Married', 1979, 3, 'Medical', 'Yes', 'travel_rarely', 15),
 (39, 'M', 'Single', 1984, 2, 'Unknown', 'No', 'travel_rarely', 33),
 (34, 'M', 'Unknown', 1989, 3, 'Unknown', 'No', 'Unknown', 9),
 (38, 'F', 'Unknown', 1985, 2, 'Life Sciences', 'No', 'Unknown', 2),
 (36, 'F', 'Married', 1987, 3, 'Marketing', 'No', 'travel_frequently', 12),
 (35, 'F', 'Unknown', 1988, 3, 'Unknown', 'No', 'Unknown', 1),
 (34, 'M', 'Unknown', 1989, 3, 'Unknown', 'No', 'Unknown', 29),
 (43, 'F', 'Single', 1980, 3, 'Medical', 'No', 'Unknown', 6),
 (42, 'F', 'Married', 1981, 3, 'Human Resources', 'No', 'travel_rarely', 10),
 (26, 'M', 'Single', 1997, 3, 'Unknown', 'No', 'travel_rarely', 7),
 (29, 'F', 'Single', 1994, 1, 'Other', 'Yes', 'Unknown', 14),
 (54, 'F', 'Unknown', 1969, 3, 'Unknown', 'No', 'Unknown', 1),
 (55, 'M', 'Divorced', 1968, 4, 'Unknown', 'No', 'Unknown', 1),
 (46, 'M', 'Divorced', 1977, 3, 'Marketing', 'Yes', 'Unknown', 9),
 (45, 'M', 'Unknown', 1978, 3, 'Medical', 'No', 'travel_rarely', 20),
 (49, 'M', 'Divorced', 1974, 4, 'Life Sciences', 'No', 'travel_rarely', 7),
 (29, 'M', 'Unknown', 1994, 3, 'Unknown', 'No', 'travel_rarely', 19),
 (29, 'M', 'Single', 1994, 4, 'Technical Degree', 'Yes', 'Unknown', 1),
 (29, 'F', 'Unknown', 1994, 1, 'Other', 'No', 'Unknown', 8),
 (53, 'F', 'Married', 1970, 5, 'Unknown', 'Yes', 'Unknown', 2),
 (44, 'F', 'Unknown', 1979, 3, 'Medical', 'No', 'Unknown', 7),
 (45, 'F', 'Unknown', 1978, 5, 'Unknown', 'No', 'Unknown', 5),
 (45, 'M', 'Unknown', 1978, 3, 'Marketing', 'No', 'non-travel', 26),
 (34, 'M', 'Unknown', 1989, 4, 'Unknown', 'No', 'travel_frequently', 10),
 (27, 'M', 'Divorced', 1996, 3, 'Unknown', 'No', 'Unknown', 36),
 (45, 'F', 'Divorced', 1978, 2, 'Unknown', 'No', 'travel_rarely', 2),
 (44, 'M', 'Married', 1979, 5, 'Unknown', 'No', 'Unknown', 1),
 (25, 'M', 'Divorced', 1998, 1, 'Life Sciences', 'No', 'Unknown', 2),
 (48, 'F', 'Unknown', 1975, 2, 'Unknown', 'Yes', 'travel_frequently', 7),
 (26, 'M', 'Unknown', 1997, 3, 'Medical', 'No', 'Unknown', 7),
 (44, 'F', 'Unknown', 1979, 3, 'Life Sciences', 'No', 'travel_rarely', 2),
 (46, 'M', 'Single', 1977, 4, 'Unknown', 'No', 'Unknown', 13),
 (35, 'M', 'Married', 1988, 4, 'Life Sciences', 'No', 'Unknown', 27),
 (30, 'F', 'Unknown', 1993, 4, 'Unknown', 'No', 'travel_frequently', 2),
 (30, 'M', 'Unknown', 1993, 2, 'Marketing', 'No', 'Unknown', 15),
 (33, 'F', 'Unknown', 1990, 3, 'Marketing', 'No', 'non-travel', 2),
 (32, 'M', 'Single', 1991, 4, 'Unknown', 'No', 'Unknown', 5),
 (43, 'M', 'Unknown', 1980, 3, 'Life Sciences', 'No', 'travel_rarely', 2),
 (35, 'M', 'Unknown', 1988, 3, 'Unknown', 'Yes', 'Unknown', 27),
 (35, 'M', 'Divorced', 1988, 2, 'Medical', 'No', 'non-travel', 2),
 (30, 'F', 'Single', 1993, 3, 'Medical', 'No', 'Unknown', 10),
 (46, 'F', 'Married', 1977, 2, 'Medical', 'Yes', 'travel_rarely', 19),
 (43, 'M', 'Married', 1980, 4, 'Life Sciences', 'No', 'travel_frequently', 2),
 (46, 'F', 'Unknown', 1977, 3, 'Life Sciences', 'Yes', 'Unknown', 10),
 (27, 'F', 'Single', 1996, 2, 'Unknown', 'No', 'travel_rarely', 2),
 (42, 'M', 'Married', 1981, 3, 'Medical', 'No', 'Unknown', 6),
 (28, 'M', 'Married', 1995, 2, 'Life Sciences', 'No', 'travel_frequently', 1),
 (47, 'M', 'Married', 1976, 4, 'Unknown', 'No', 'Unknown', 14),
 (34, 'M', 'Divorced', 1989, 4, 'Medical', 'No', 'travel_rarely', 13),
 (26, 'M', 'Single', 1997, 3, 'Medical', 'No', 'Unknown', 1),
 (27, 'M', 'Unknown', 1996, 3, 'Unknown', 'No', 'travel_rarely', 9),
 (31, 'M', 'Unknown', 1992, 4, 'Marketing', 'Yes', 'travel_frequently', 26),
 (31, 'F', 'Married', 1992, 3, 'Unknown', 'No', 'Unknown', 10),
 (35, 'M', 'Unknown', 1988, 4, 'Medical', 'No', 'Unknown', 9),
 (33, 'F', 'Married', 1990, 2, 'Unknown', 'No', 'travel_rarely', 15),
 (39, 'M', 'Single', 1984, 1, 'Life Sciences', 'No', 'travel_rarely', 24),
 (41, 'F', 'Married', 1982, 4, 'Life Sciences', 'No', 'travel_rarely', 28),
 (34, 'F', 'Married', 1989, 3, 'Marketing', 'No', 'Unknown', 28),
 (39, 'M', 'Unknown', 1984, 1, 'Unknown', 'No', 'travel_rarely', 6),
 (34, 'F', 'Single', 1989, 4, 'Unknown', 'No', 'travel_rarely', 7),
 (46, 'M', 'Single', 1977, 4, 'Unknown', 'No', 'Unknown', 5),
 (41,
  'M',
  'Divorced',
  1982,
  3,
  'Life Sciences',
  'No',
  'travel_frequently',
  10),
 (39, 'F', 'Single', 1984, 4, 'Unknown', 'No', 'Unknown', 2),
 (42, 'F', 'Divorced', 1981, 3, 'Medical', 'No', 'travel_rarely', 2),
 (33, 'F', 'Married', 1990, 4, 'Unknown', 'No', 'travel_frequently', 5),
 (35, 'M', 'Single', 1988, 4, 'Technical Degree', 'No', 'travel_rarely', 5),
 (41, 'M', 'Divorced', 1982, 4, 'Human Resources', 'No', 'travel_rarely', 10),
 (27, 'F', 'Unknown', 1996, 3, 'Unknown', 'No', 'Unknown', 7),
 (51, 'F', 'Married', 1972, 4, 'Marketing', 'No', 'travel_rarely', 26),
 (47, 'F', 'Unknown', 1976, 2, 'Marketing', 'No', 'travel_rarely', 2),
 (50, 'M', 'Divorced', 1973, 4, 'Unknown', 'No', 'non-travel', 2),
 (35, 'F', 'Married', 1988, 1, 'Life Sciences', 'No', 'travel_rarely', 21),
 (39, 'F', 'Married', 1984, 1, 'Unknown', 'No', 'travel_rarely', 1),
 (41, 'F', 'Unknown', 1982, 2, 'Unknown', 'No', 'Unknown', 7),
 (42, 'M', 'Unknown', 1981, 2, 'Medical', 'No', 'Unknown', 2),
 (52, 'M', 'Single', 1971, 4, 'Life Sciences', 'No', 'travel_rarely', 7),
 (38, 'M', 'Unknown', 1985, 3, 'Unknown', 'No', 'Unknown', 1),
 (35, 'M', 'Single', 1988, 4, 'Life Sciences', 'No', 'Unknown', 4),
 (28, 'M', 'Married', 1995, 1, 'Life Sciences', 'No', 'travel_rarely', 29),
 (44, 'M', 'Married', 1979, 3, 'Life Sciences', 'No', 'Unknown', 28),
 (50, 'M', 'Unknown', 1973, 5, 'Unknown', 'No', 'Unknown', 22),
 (39, 'M', 'Married', 1984, 3, 'Unknown', 'No', 'Unknown', 6),
 (49, 'F', 'Unknown', 1974, 4, 'Unknown', 'No', 'Unknown', 13),
 (45, 'F', 'Single', 1978, 2, 'Technical Degree', 'No', 'Unknown', 28),
 (27, 'F', 'Married', 1996, 3, 'Unknown', 'No', 'Unknown', 15),
 (42, 'M', 'Married', 1981, 3, 'Unknown', 'No', 'travel_rarely', 2),
 (49, 'M', 'Married', 1974, 3, 'Unknown', 'No', 'Unknown', 2),
 (29, 'F', 'Single', 1994, 2, 'Life Sciences', 'No', 'travel_rarely', 15),
 (34, 'M', 'Unknown', 1989, 4, 'Marketing', 'No', 'non-travel', 12),
 (36, 'M', 'Single', 1987, 3, 'Unknown', 'No', 'Unknown', 11),
 (29, 'M', 'Unknown', 1994, 3, 'Unknown', 'No', 'travel_rarely', 15),
 (26, 'F', 'Unknown', 1997, 3, 'Unknown', 'No', 'Unknown', 1),
 (28, 'M', 'Single', 1995, 4, 'Life Sciences', 'No', 'Unknown', 9),
 (42,
  'F',
  'Married',
  1981,
  4,
  'Technical Degree',
  'No',
  'travel_frequently',
  30),
 (31, 'M', 'Unknown', 1992, 1, 'Life Sciences', 'No', 'travel_rarely', 2),
 (37, 'M', 'Divorced', 1986, 4, 'Unknown', 'No', 'non-travel', 9),
 (50, 'M', 'Married', 1973, 3, 'Unknown', 'No', 'Unknown', 3),
 (34, 'M', 'Unknown', 1989, 2, 'Life Sciences', 'No', 'Unknown', 1),
 (27, 'F', 'Unknown', 1996, 1, 'Medical', 'No', 'travel_frequently', 3),
 (25, 'F', 'Single', 1998, 1, 'Unknown', 'No', 'Unknown', 42),
 (34, 'F', 'Unknown', 1989, 4, 'Unknown', 'No', 'Unknown', 2),
 (35, 'F', 'Unknown', 1988, 1, 'Medical', 'No', 'Unknown', 16),
 (33, 'F', 'Married', 1990, 4, 'Unknown', 'No', 'Unknown', 1),
 (29, 'F', 'Divorced', 1994, 4, 'Life Sciences', 'No', 'Unknown', 28),
 (29, 'F', 'Divorced', 1994, 3, 'Unknown', 'No', 'travel_frequently', 10),
 (31, 'M', 'Married', 1992, 2, 'Unknown', 'No', 'travel_rarely', 11),
 (47, 'M', 'Unknown', 1976, 3, 'Unknown', 'No', 'travel_rarely', 1),
 (32, 'M', 'Married', 1991, 4, 'Unknown', 'No', 'Unknown', 1),
 (32, 'M', 'Divorced', 1991, 3, 'Medical', 'No', 'Unknown', 2),
 (29, 'M', 'Married', 1994, 3, 'Life Sciences', 'No', 'travel_rarely', 2),
 (56, 'M', 'Unknown', 1967, 3, 'Unknown', 'No', 'Unknown', 13),
 (28, 'M', 'Unknown', 1995, 3, 'Medical', 'No', 'Unknown', 4),
 (25, 'M', 'Single', 1998, 3, 'Unknown', 'Yes', 'travel_rarely', 5),
 (56, 'F', 'Unknown', 1967, 4, 'Life Sciences', 'Yes', 'Unknown', 14),
 (26, 'F', 'Married', 1997, 3, 'Unknown', 'No', 'Unknown', 6),
 (32, 'F', 'Unknown', 1991, 3, 'Medical', 'Yes', 'Unknown', 9),
 (30, 'F', 'Single', 1993, 3, 'Life Sciences', 'No', 'travel_rarely', 18),
 (50, 'F', 'Unknown', 1973, 3, 'Unknown', 'No', 'travel_rarely', 11),
 (32, 'M', 'Unknown', 1991, 5, 'Life Sciences', 'No', 'Unknown', 6),
 (45, 'F', 'Unknown', 1978, 2, 'Unknown', 'No', 'travel_rarely', 4),
 (43, 'M', 'Divorced', 1980, 3, 'Unknown', 'No', 'Unknown', 7),
 (29, 'F', 'Married', 1994, 3, 'Marketing', 'Yes', 'Unknown', 27),
 (31, 'M', 'Unknown', 1992, 4, 'Medical', 'No', 'non-travel', 1),
 (33, 'M', 'Married', 1990, 3, 'Unknown', 'Yes', 'Unknown', 14),
 (31, 'M', 'Unknown', 1992, 2, 'Unknown', 'Yes', 'Unknown', 29),
 (37, 'M', 'Unknown', 1986, 4, 'Unknown', 'No', 'Unknown', 6),
 (27, 'M', 'Unknown', 1996, 1, 'Marketing', 'No', 'non-travel', 1),
 (41, 'M', 'Single', 1982, 3, 'Unknown', 'No', 'travel_rarely', 6),
 (37, 'F', 'Unknown', 1986, 3, 'Medical', 'No', 'travel_rarely', 10),
 (40, 'M', 'Unknown', 1983, 2, 'Unknown', 'Yes', 'Unknown', 22),
 (55, 'F', 'Single', 1968, 3, 'Unknown', 'No', 'Unknown', 2),
 (26, 'M', 'Unknown', 1997, 1, 'Unknown', 'Yes', 'travel_frequently', 3),
 (25, 'F', 'Single', 1998, 2, 'Unknown', 'No', 'Unknown', 4),
 (42, 'F', 'Unknown', 1981, 2, 'Unknown', 'No', 'travel_frequently', 12),
 (30, 'F', 'Unknown', 1993, 3, 'Life Sciences', 'No', 'travel_rarely', 2),
 (35, 'M', 'Single', 1988, 3, 'Life Sciences', 'No', 'travel_rarely', 22),
 (32, 'F', 'Married', 1991, 4, 'Unknown', 'No', 'travel_rarely', 9),
 (47, 'F', 'Unknown', 1976, 2, 'Life Sciences', 'Yes', 'Unknown', 27),
 (30, 'M', 'Divorced', 1993, 4, 'Life Sciences', 'No', 'travel_frequently', 5),
 (45, 'M', 'Divorced', 1978, 3, 'Marketing', 'No', 'Unknown', 9),
 (36, 'M', 'Unknown', 1987, 3, 'Life Sciences', 'No', 'travel_rarely', 8),
 (31, 'M', 'Unknown', 1992, 4, 'Unknown', 'No', 'Unknown', 11),
 (37, 'F', 'Married', 1986, 2, 'Medical', 'No', 'travel_frequently', 21),
 (42, 'F', 'Unknown', 1981, 2, 'Unknown', 'No', 'Unknown', 14),
 (50, 'M', 'Divorced', 1973, 2, 'Medical', 'No', 'Unknown', 1),
 (59, 'M', 'Unknown', 1964, 2, 'Unknown', 'No', 'travel_rarely', 1),
 (56, 'F', 'Married', 1967, 3, 'Medical', 'No', 'Unknown', 9),
 (48, 'M', 'Single', 1975, 4, 'Unknown', 'No', 'Unknown', 1),
 (39, 'M', 'Single', 1984, 3, 'Life Sciences', 'No', 'Unknown', 9),
 (42, 'M', 'Single', 1981, 4, 'Marketing', 'No', 'travel_rarely', 4),
 (44, 'M', 'Single', 1979, 4, 'Unknown', 'Yes', 'Unknown', 10),
 (29, 'F', 'Married', 1994, 1, 'Unknown', 'No', 'travel_rarely', 15),
 (54, 'F', 'Unknown', 1969, 4, 'Unknown', 'No', 'travel_frequently', 1),
 (46, 'F', 'Married', 1977, 2, 'Unknown', 'No', 'Unknown', 4),
 (32, 'M', 'Unknown', 1991, 2, 'Life Sciences', 'Yes', 'travel_rarely', 7),
 (34, 'F', 'Divorced', 1989, 1, 'Unknown', 'No', 'non-travel', 26),
 (39, 'M', 'Married', 1984, 4, 'Unknown', 'No', 'travel_rarely', 21),
 (33, 'M', 'Married', 1990, 1, 'Life Sciences', 'No', 'Unknown', 41),
 (39, 'M', 'Unknown', 1984, 2, 'Unknown', 'No', 'non-travel', 25),
 (52, 'F', 'Married', 1971, 4, 'Other', 'No', 'travel_rarely', 8),
 (36, 'F', 'Single', 1987, 4, 'Unknown', 'No', 'Unknown', 3),
 (36, 'M', 'Single', 1987, 4, 'Life Sciences', 'No', 'Unknown', 47),
 (24, 'F', 'Married', 1999, 1, 'Unknown', 'No', 'travel_rarely', 4),
 (28, 'M', 'Single', 1995, 3, 'Unknown', 'No', 'Unknown', 5),
 (47, 'F', 'Married', 1976, 3, 'Medical', 'No', 'travel_rarely', 14),
 (40, 'M', 'Single', 1983, 4, 'Life Sciences', 'Yes', 'Unknown', 9),
 (45, 'M', 'Married', 1978, 3, 'Technical Degree', 'No', 'travel_rarely', 49),
 (40, 'M', 'Unknown', 1983, 2, 'Unknown', 'No', 'travel_rarely', 9),
 (40, 'F', 'Married', 1983, 2, 'Life Sciences', 'No', 'Unknown', 14),
 (43, 'M', 'Divorced', 1980, 3, 'Unknown', 'No', 'Unknown', 27),
 (30, 'M', 'Unknown', 1993, 3, 'Unknown', 'No', 'Unknown', 7),
 (38, 'F', 'Unknown', 1985, 2, 'Unknown', 'No', 'Unknown', 2),
 (37, 'F', 'Married', 1986, 3, 'Unknown', 'No', 'travel_rarely', 10),
 (36, 'M', 'Divorced', 1987, 2, 'Life Sciences', 'No', 'travel_rarely', 11),
 (23, 'M', 'Single', 2000, 1, 'Medical', 'Yes', 'travel_rarely', 8),
 (35, 'M', 'Unknown', 1988, 4, 'Unknown', 'No', 'Unknown', 2),
 (31,
  'F',
  'Single',
  1992,
  1,
  'Technical Degree',
  'No',
  'travel_frequently',
  24),
 (38, 'F', 'Married', 1985, 2, 'Other', 'No', 'travel_rarely', 6),
 (35, 'M', 'Married', 1988, 4, 'Medical', 'No', 'Unknown', 23),
 (27, 'F', 'Single', 1996, 2, 'Unknown', 'No', 'travel_frequently', 48),
 (41, 'F', 'Unknown', 1982, 3, 'Unknown', 'No', 'Unknown', 8),
 (33, 'M', 'Single', 1990, 4, 'Marketing', 'No', 'travel_rarely', 15),
 (49, 'M', 'Married', 1974, 1, 'Life Sciences', 'No', 'Unknown', 2),
 (36, 'M', 'Unknown', 1987, 4, 'Unknown', 'No', 'non-travel', 9),
 (36, 'M', 'Single', 1987, 4, 'Life Sciences', 'Yes', 'travel_rarely', 9),
 (43, 'F', 'Unknown', 1980, 2, 'Medical', 'No', 'travel_rarely', 2),
 (33, 'F', 'Married', 1990, 3, 'Life Sciences', 'No', 'travel_frequently', 18),
 (44, 'F', 'Unknown', 1979, 5, 'Marketing', 'No', 'Unknown', 1),
 (38, 'F', 'Single', 1985, 5, 'Life Sciences', 'No', 'Unknown', 29),
 (50, 'F', 'Divorced', 1973, 2, 'Medical', 'No', 'Unknown', 7),
 (59, 'F', 'Single', 1964, 1, 'Unknown', 'No', 'travel_frequently', 1),
 (40, 'M', 'Married', 1983, 4, 'Life Sciences', 'No', 'travel_frequently', 1),
 (54, 'F', 'Unknown', 1969, 4, 'Technical Degree', 'No', 'travel_rarely', 2),
 (54, 'F', 'Single', 1969, 3, 'Unknown', 'No', 'non-travel', 26),
 (24, 'M', 'Married', 1999, 1, 'Life Sciences', 'No', 'Unknown', 18),
 (30, 'M', 'Single', 1993, 1, 'Unknown', 'No', 'travel_rarely', 2),
 (34, 'F', 'Unknown', 1989, 4, 'Unknown', 'No', 'travel_frequently', 10),
 (38, 'F', 'Single', 1985, 4, 'Life Sciences', 'No', 'travel_rarely', 1),
 (42, 'F', 'Divorced', 1981, 2, 'Unknown', 'No', 'Unknown', 21),
 (34, 'M', 'Divorced', 1989, 4, 'Unknown', 'No', 'Unknown', 10),
 (30, 'F', 'Unknown', 1993, 4, 'Unknown', 'Yes', 'Unknown', 26),
 (29, 'M', 'Single', 1994, 1, 'Medical', 'Yes', 'travel_rarely', 13),
 (32, 'F', 'Married', 1991, 2, 'Unknown', 'No', 'Unknown', 4),
 (25, 'M', 'Unknown', 1998, 2, 'Life Sciences', 'No', 'Unknown', 5),
 (53, 'M', 'Married', 1970, 4, 'Unknown', 'No', 'Unknown', 3),
 (26, 'M', 'Married', 1997, 2, 'Medical', 'No', 'Unknown', 11),
 (28, 'M', 'Single', 1995, 2, 'Medical', 'No', 'travel_rarely', 16),
 (58, 'M', 'Divorced', 1965, 4, 'Life Sciences', 'No', 'Unknown', 1),
 (27, 'F', 'Single', 1996, 3, 'Unknown', 'No', 'travel_rarely', 5),
 (54, 'F', 'Unknown', 1969, 4, 'Life Sciences', 'No', 'travel_frequently', 20),
 (29, 'M', 'Unknown', 1994, 3, 'Life Sciences', 'No', 'non-travel', 2),
 (50, 'M', 'Unknown', 1973, 3, 'Unknown', 'No', 'travel_rarely', 10),
 (24, 'F', 'Single', 1999, 1, 'Technical Degree', 'Yes', 'travel_rarely', 1),
 (34, 'M', 'Unknown', 1989, 4, 'Unknown', 'No', 'Unknown', 2),
 (36, 'M', 'Single', 1987, 4, 'Medical', 'No', 'non-travel', 30),
 (41, 'M', 'Single', 1982, 2, 'Unknown', 'No', 'non-travel', 10),
 (26, 'M', 'Married', 1997, 2, 'Marketing', 'No', 'travel_rarely', 28),
 (27, 'F', 'Divorced', 1996, 4, 'Unknown', 'No', 'Unknown', 6),
 (27, 'M', 'Divorced', 1996, 1, 'Marketing', 'Yes', 'travel_rarely', 2),
 (23, 'M', 'Unknown', 2000, 1, 'Unknown', 'No', 'travel_rarely', 49),
 (39, 'F', 'Unknown', 1984, 3, 'Unknown', 'No', 'travel_rarely', 24),
 (52, 'M', 'Unknown', 1971, 4, 'Medical', 'No', 'non-travel', 19),
 (26, 'M', 'Single', 1997, 3, 'Unknown', 'Yes', 'Unknown', 8),
 (54, 'F', 'Divorced', 1969, 4, 'Unknown', 'No', 'Unknown', 19),
 (50, 'M', 'Unknown', 1973, 3, 'Unknown', 'No', 'travel_rarely', 2),
 (42, 'F', 'Married', 1981, 2, 'Life Sciences', 'No', 'Unknown', 1),
 (32, 'F', 'Married', 1991, 2, 'Unknown', 'No', 'Unknown', 8),
 (42, 'F', 'Married', 1981, 3, 'Unknown', 'No', 'Unknown', 3),
 (38, 'F', 'Unknown', 1985, 3, 'Unknown', 'No', 'non-travel', 10),
 (45, 'F', 'Divorced', 1978, 4, 'Unknown', 'No', 'travel_rarely', 1),
 (31, 'F', 'Unknown', 1992, 1, 'Unknown', 'No', 'travel_frequently', 1),
 (22, 'M', 'Unknown', 2001, 1, 'Life Sciences', 'No', 'Unknown', 8),
 (59, 'F', 'Unknown', 1964, 3, 'Unknown', 'No', 'travel_rarely', 2),
 (45, 'F', 'Married', 1978, 2, 'Unknown', 'No', 'Unknown', 11),
 (45, 'M', 'Unknown', 1978, 4, 'Unknown', 'No', 'travel_rarely', 24),
 (26, 'F', 'Unknown', 1997, 1, 'Medical', 'No', 'Unknown', 26),
 (28, 'M', 'Married', 1995, 1, 'Unknown', 'No', 'Unknown', 42),
 (30, 'M', 'Married', 1993, 3, 'Marketing', 'No', 'travel_rarely', 10),
 (43, 'F', 'Single', 1980, 3, 'Unknown', 'Yes', 'Unknown', 9),
 (30, 'M', 'Divorced', 1993, 4, 'Life Sciences', 'No', 'travel_rarely', 29),
 (46, 'M', 'Divorced', 1977, 4, 'Technical Degree', 'No', 'travel_rarely', 2),
 (42, 'M', 'Married', 1981, 5, 'Unknown', 'No', 'non-travel', 2),
 (27, 'M', 'Divorced', 1996, 2, 'Unknown', 'No', 'Unknown', 4),
 (56, 'M', 'Single', 1967, 4, 'Life Sciences', 'No', 'travel_rarely', 8),
 (41, 'F', 'Unknown', 1982, 3, 'Marketing', 'No', 'travel_frequently', 1),
 (28, 'M', 'Married', 1995, 3, 'Life Sciences', 'No', 'Unknown', 26),
 (52, 'F', 'Married', 1971, 2, 'Unknown', 'No', 'travel_frequently', 28),
 (26, 'M', 'Single', 1997, 1, 'Medical', 'No', 'travel_rarely', 2),
 (49, 'M', 'Unknown', 1974, 2, 'Unknown', 'Yes', 'travel_frequently', 28),
 (34, 'M', 'Unknown', 1989, 4, 'Other', 'No', 'travel_frequently', 1),
 (32, 'M', 'Unknown', 1991, 5, 'Marketing', 'No', 'Unknown', 7),
 (33, 'M', 'Unknown', 1990, 3, 'Medical', 'No', 'Unknown', 7),
 (29, 'F', 'Single', 1994, 3, 'Unknown', 'Yes', 'Unknown', 13),
 (30, 'M', 'Unknown', 1993, 3, 'Life Sciences', 'No', 'Unknown', 16),
 (23, 'M', 'Single', 2000, 1, 'Life Sciences', 'No', 'Unknown', 11),
 (33, 'M', 'Unknown', 1990, 3, 'Unknown', 'No', 'Unknown', 25),
 (35, 'F', 'Married', 1988, 4, 'Unknown', 'No', 'Unknown', 28),
 (38, 'M', 'Unknown', 1985, 4, 'Unknown', 'No', 'travel_frequently', 10),
 (28, 'F', 'Divorced', 1995, 3, 'Unknown', 'No', 'Unknown', 27),
 (37, 'M', 'Unknown', 1986, 3, 'Life Sciences', 'No', 'Unknown', 19),
 (24, 'F', 'Unknown', 1999, 1, 'Technical Degree', 'No', 'Unknown', 2),
 (36, 'M', 'Unknown', 1987, 3, 'Unknown', 'No', 'Unknown', 7),
 (53, 'M', 'Married', 1970, 4, 'Medical', 'No', 'travel_rarely', 24),
 (37, 'M', 'Unknown', 1986, 4, 'Unknown', 'No', 'travel_frequently', 1),
 (32, 'M', 'Married', 1991, 3, 'Unknown', 'No', 'travel_rarely', 19),
 (42, 'M', 'Divorced', 1981, 3, 'Medical', 'No', 'travel_rarely', 12),
 (41, 'M', 'Divorced', 1982, 4, 'Life Sciences', 'No', 'Unknown', 9),
 (44, 'M', 'Unknown', 1979, 2, 'Unknown', 'Yes', 'Unknown', 1),
 (39, 'M', 'Single', 1984, 1, 'Medical', 'No', 'Unknown', 10),
 (40, 'F', 'Unknown', 1983, 3, 'Unknown', 'No', 'Unknown', 1),
 (42, 'M', 'Married', 1981, 4, 'Technical Degree', 'No', 'Unknown', 10),
 (34, 'M', 'Married', 1989, 3, 'Unknown', 'No', 'travel_rarely', 4),
 (37, 'M', 'Unknown', 1986, 4, 'Medical', 'No', 'travel_rarely', 7),
 (27, 'M', 'Unknown', 1996, 4, 'Technical Degree', 'No', 'Unknown', 2),
 (35, 'M', 'Unknown', 1988, 4, 'Unknown', 'No', 'travel_rarely', 18),
 (28, 'M', 'Single', 1995, 3, 'Medical', 'No', 'non-travel', 16),
 (29, 'M', 'Married', 1994, 3, 'Life Sciences', 'No', 'Unknown', 1),
 (54, 'F', 'Married', 1969, 2, 'Unknown', 'No', 'Unknown', 9),
 (24, 'M', 'Single', 1999, 2, 'Life Sciences', 'No', 'travel_rarely', 41),
 (38, 'M', 'Married', 1985, 4, 'Unknown', 'No', 'travel_frequently', 6),
 (42, 'M', 'Unknown', 1981, 4, 'Unknown', 'No', 'Unknown', 18),
 (41, 'M', 'Unknown', 1982, 3, 'Life Sciences', 'No', 'Unknown', 1),
 (25, 'F', 'Unknown', 1998, 2, 'Unknown', 'No', 'travel_rarely', 8),
 (50, 'M', 'Unknown', 1973, 4, 'Unknown', 'Yes', 'Unknown', 1),
 (45, 'F', 'Unknown', 1978, 3, 'Unknown', 'No', 'Unknown', 20),
 (35, 'F', 'Married', 1988, 4, 'Unknown', 'No', 'travel_rarely', 1),
 (29, 'M', 'Unknown', 1994, 2, 'Other', 'No', 'travel_rarely', 13),
 (34, 'M', 'Unknown', 1989, 3, 'Unknown', 'No', 'Unknown', 8),
 (27, 'M', 'Married', 1996, 1, 'Unknown', 'No', 'travel_rarely', 2),
 (31, 'M', 'Divorced', 1992, 1, 'Unknown', 'No', 'Unknown', 26),
 (34, 'M', 'Divorced', 1989, 2, 'Unknown', 'No', 'travel_rarely', 38),
 (28, 'M', 'Single', 1995, 3, 'Life Sciences', 'Yes', 'travel_rarely', 24),
 (32, 'M', 'Divorced', 1991, 2, 'Unknown', 'No', 'travel_rarely', 5),
 (38, 'M', 'Single', 1985, 3, 'Life Sciences', 'No', 'Unknown', 2),
 (24, 'F', 'Unknown', 1999, 2, 'Unknown', 'No', 'Unknown', 11),
 (34, 'M', 'Single', 1989, 1, 'Unknown', 'Yes', 'Unknown', 6),
 (32, 'F', 'Unknown', 1991, 1, 'Unknown', 'Yes', 'travel_frequently', 16),
 (46, 'F', 'Single', 1977, 4, 'Marketing', 'No', 'travel_rarely', 2),
 (44, 'M', 'Married', 1979, 4, 'Other', 'No', 'Unknown', 10),
 (50, 'M', 'Married', 1973, 2, 'Unknown', 'Yes', 'travel_rarely', 3),
 (35, 'F', 'Married', 1988, 3, 'Marketing', 'No', 'Unknown', 2),
 (28, 'M', 'Single', 1995, 4, 'Unknown', 'Yes', 'Unknown', 5),
 (26, 'F', 'Married', 1997, 3, 'Unknown', 'No', 'Unknown', 22),
 (35, 'M', 'Married', 1988, 3, 'Medical', 'No', 'Unknown', 31),
 (37, 'M', 'Unknown', 1986, 4, 'Medical', 'No', 'travel_rarely', 20),
 (46, 'M', 'Divorced', 1977, 2, 'Medical', 'No', 'travel_rarely', 38),
 (34, 'M', 'Married', 1989, 2, 'Medical', 'No', 'travel_rarely', 1),
 (51, 'M', 'Unknown', 1972, 4, 'Unknown', 'No', 'travel_rarely', 9),
 (24, 'F', 'Married', 1999, 2, 'Other', 'No', 'travel_rarely', 3),
 (44, 'M', 'Married', 1979, 3, 'Unknown', 'No', 'Unknown', 23),
 (37, 'M', 'Divorced', 1986, 4, 'Human Resources', 'Yes', 'travel_rarely', 6),
 (25, 'M', 'Married', 1998, 1, 'Unknown', 'No', 'Unknown', 7),
 (43, 'F', 'Single', 1980, 3, 'Medical', 'No', 'Unknown', 26),
 (31, 'F', 'Unknown', 1992, 3, 'Unknown', 'Yes', 'travel_rarely', 20),
 (36, 'M', 'Divorced', 1987, 2, 'Medical', 'No', 'Unknown', 6),
 (51, 'M', 'Unknown', 1972, 4, 'Unknown', 'Yes', 'travel_frequently', 16),
 (38, 'F', 'Unknown', 1985, 5, 'Technical Degree', 'No', 'travel_rarely', 3),
 (30, 'F', 'Divorced', 1993, 3, 'Technical Degree', 'No', 'travel_rarely', 5),
 (27, 'M', 'Unknown', 1996, 3, 'Medical', 'No', 'non-travel', 9),
 (28, 'M', 'Married', 1995, 3, 'Medical', 'No', 'Unknown', 21),
 (27, 'M', 'Unknown', 1996, 3, 'Unknown', 'No', 'Unknown', 17),
 (56, 'F', 'Divorced', 1967, 3, 'Life Sciences', 'No', 'travel_rarely', 8),
 (38, 'F', 'Single', 1985, 2, 'Unknown', 'No', 'travel_rarely', 9),
 (31, 'F', 'Unknown', 1992, 1, 'Medical', 'No', 'travel_rarely', 12),
 (34, 'F', 'Unknown', 1989, 2, 'Medical', 'No', 'travel_rarely', 27),
 (50, 'M', 'Unknown', 1973, 3, 'Unknown', 'No', 'Unknown', 1),
 (53, 'F', 'Divorced', 1970, 2, 'Unknown', 'No', 'Unknown', 13),
 (35, 'F', 'Single', 1988, 3, 'Medical', 'No', 'travel_frequently', 24),
 (40, 'M', 'Unknown', 1983, 5, 'Life Sciences', 'No', 'Unknown', 10),
 (46, 'M', 'Unknown', 1977, 4, 'Unknown', 'No', 'Unknown', 1),
 (30, 'F', 'Unknown', 1993, 3, 'Technical Degree', 'Yes', 'Unknown', 3),
 (54, 'F', 'Married', 1969, 3, 'Unknown', 'No', 'Unknown', 3),
 (33, 'F', 'Married', 1990, 4, 'Unknown', 'No', 'travel_rarely', 4),
 (52, 'F', 'Single', 1971, 3, 'Unknown', 'No', 'travel_rarely', 2),
 (39, 'F', 'Divorced', 1984, 4, 'Medical', 'No', 'Unknown', 24),
 (31, 'M', 'Divorced', 1992, 4, 'Medical', 'No', 'Unknown', 3),
 (33, 'F', 'Married', 1990, 4, 'Life Sciences', 'No', 'travel_rarely', 5),
 (45, 'M', 'Unknown', 1978, 4, 'Life Sciences', 'No', 'Unknown', 23),
 (42, 'F', 'Single', 1981, 2, 'Unknown', 'No', 'Unknown', 17),
 (29, 'M', 'Divorced', 1994, 2, 'Marketing', 'No', 'Unknown', 19),
 (25, 'F', 'Married', 1998, 3, 'Unknown', 'No', 'travel_rarely', 1),
 (32, 'M', 'Unknown', 1991, 1, 'Unknown', 'No', 'Unknown', 2),
 (21, 'M', 'Single', 2002, 2, 'Unknown', 'No', 'travel_rarely', 3),
 (37, 'F', 'Divorced', 1986, 4, 'Medical', 'No', 'non-travel', 1),
 (38, 'M', 'Unknown', 1985, 4, 'Life Sciences', 'No', 'travel_rarely', 23),
 (26, 'M', 'Divorced', 1997, 4, 'Medical', 'Yes', 'travel_rarely', 16),
 (49, 'F', 'Single', 1974, 4, 'Life Sciences', 'No', 'Unknown', 10),
 (37, 'M', 'Married', 1986, 3, 'Medical', 'No', 'travel_frequently', 9),
 (31, 'M', 'Married', 1992, 4, 'Unknown', 'No', 'travel_rarely', 11),
 (31, 'M', 'Unknown', 1992, 3, 'Human Resources', 'No', 'travel_rarely', 2),
 (52, 'M', 'Unknown', 1971, 4, 'Medical', 'Yes', 'Unknown', 17),
 (28, 'M', 'Married', 1995, 4, 'Medical', 'Yes', 'Unknown', 2),
 (47, 'M', 'Married', 1976, 5, 'Unknown', 'No', 'travel_rarely', 5),
 (29, 'M', 'Unknown', 1994, 5, 'Other', 'No', 'Unknown', 9),
 (37, 'M', 'Unknown', 1986, 2, 'Medical', 'No', 'Unknown', 5),
 (23, 'M', 'Unknown', 2000, 1, 'Unknown', 'No', 'Unknown', 26),
 (42, 'M', 'Divorced', 1981, 1, 'Unknown', 'No', 'travel_rarely', 2),
 (30, 'F', 'Unknown', 1993, 2, 'Unknown', 'No', 'Unknown', 25),
 (21, 'F', 'Single', 2002, 1, 'Unknown', 'Yes', 'Unknown', 31),
 (57, 'M', 'Unknown', 1966, 4, 'Medical', 'No', 'travel_rarely', 1),
 (29, 'M', 'Unknown', 1994, 3, 'Unknown', 'No', 'Unknown', 17),
 (36, 'F', 'Married', 1987, 3, 'Unknown', 'No', 'travel_rarely', 2),
 (34, 'M', 'Unknown', 1989, 3, 'Marketing', 'Yes', 'non-travel', 19),
 (24, 'F', 'Divorced', 1999, 4, 'Marketing', 'No', 'Unknown', 10),
 (42, 'F', 'Unknown', 1981, 4, 'Unknown', 'No', 'Unknown', 1),
 (54, 'M', 'Married', 1969, 2, 'Medical', 'No', 'Unknown', 5),
 (34, 'F', 'Married', 1989, 4, 'Unknown', 'No', 'Unknown', 4),
 (43, 'M', 'Unknown', 1980, 3, 'Unknown', 'No', 'Unknown', 21),
 (43, 'F', 'Unknown', 1980, 4, 'Other', 'No', 'travel_rarely', 8),
 (31, 'M', 'Married', 1992, 2, 'Unknown', 'No', 'travel_rarely', 1),
 (45, 'M', 'Married', 1978, 2, 'Life Sciences', 'No', 'Unknown', 10),
 (34, 'F', 'Unknown', 1989, 2, 'Unknown', 'Yes', 'travel_frequently', 6),
 (29, 'M', 'Unknown', 1994, 4, 'Unknown', 'No', 'Unknown', 3),
 (46, 'F', 'Unknown', 1977, 3, 'Marketing', 'No', 'Unknown', 10),
 (30, 'M', 'Single', 1993, 1, 'Marketing', 'No', 'travel_rarely', 7),
 (42, 'M', 'Married', 1981, 2, 'Other', 'No', 'Unknown', 5),
 (22, 'M', 'Single', 2001, 1, 'Technical Degree', 'Yes', 'Unknown', 25),
 (29, 'M', 'Divorced', 1994, 3, 'Technical Degree', 'No', 'travel_rarely', 19),
 (46, 'F', 'Married', 1977, 1, 'Medical', 'No', 'travel_frequently', 18),
 (32, 'M', 'Unknown', 1991, 4, 'Other', 'Yes', 'Unknown', 11),
 (24, 'M', 'Unknown', 1999, 3, 'Medical', 'No', 'Unknown', 24),
 (35, 'M', 'Single', 1988, 2, 'Unknown', 'No', 'travel_rarely', 25),
 (42, 'F', 'Married', 1981, 4, 'Life Sciences', 'No', 'Unknown', 2),
 (48, 'F', 'Married', 1975, 4, 'Unknown', 'No', 'travel_rarely', 1),
 (44, 'F', 'Unknown', 1979, 4, 'Unknown', 'No', 'travel_rarely', 1),
 (34, 'F', 'Unknown', 1989, 3, 'Unknown', 'No', 'Unknown', 14),
 (44, 'M', 'Unknown', 1979, 4, 'Unknown', 'No', 'travel_rarely', 3),
 (22, 'M', 'Married', 2001, 1, 'Medical', 'No', 'travel_rarely', 6),
 (26, 'F', 'Unknown', 1997, 3, 'Unknown', 'No', 'travel_rarely', 6),
 (36, 'M', 'Unknown', 1987, 2, 'Medical', 'No', 'Unknown', 9),
 (23, 'M', 'Unknown', 2000, 3, 'Unknown', 'No', 'travel_rarely', 4),
 (25, 'M', 'Unknown', 1998, 3, 'Life Sciences', 'No', 'Unknown', 8),
 (48, 'M', 'Married', 1975, 1, 'Unknown', 'No', 'Unknown', 14),
 (60, 'M', 'Single', 1963, 4, 'Unknown', 'No', 'travel_rarely', 16),
 (50, 'M', 'Married', 1973, 4, 'Marketing', 'No', 'Unknown', 8),
 (44, 'M', 'Single', 1979, 3, 'Unknown', 'No', 'non-travel', 24),
 (47, 'F', 'Divorced', 1976, 3, 'Unknown', 'No', 'travel_frequently', 3),
 (42, 'M', 'Divorced', 1981, 3, 'Unknown', 'Yes', 'Unknown', 19),
 (43, 'M', 'Married', 1980, 3, 'Life Sciences', 'No', 'travel_frequently', 15),
 (53, 'M', 'Single', 1970, 3, 'Unknown', 'No', 'Unknown', 6),
 (26, 'F', 'Unknown', 1997, 2, 'Life Sciences', 'No', 'travel_frequently', 1),
 (34, 'F', 'Single', 1989, 4, 'Unknown', 'No', 'travel_rarely', 1),
 (27, 'M', 'Married', 1996, 1, 'Marketing', 'No', 'Unknown', 8),
 (30, 'M', 'Married', 1993, 3, 'Unknown', 'No', 'travel_rarely', 18),
 (52, 'F', 'Unknown', 1971, 4, 'Life Sciences', 'No', 'travel_rarely', 25),
 (33, 'F', 'Single', 1990, 3, 'Marketing', 'Yes', 'travel_rarely', 16),
 (34, 'F', 'Single', 1989, 4, 'Unknown', 'No', 'travel_frequently', 16),
 (34, 'M', 'Unknown', 1989, 2, 'Unknown', 'No', 'Unknown', 3),
 (44, 'M', 'Single', 1979, 3, 'Unknown', 'No', 'travel_rarely', 4),
 (32, 'M', 'Divorced', 1991, 4, 'Life Sciences', 'No', 'Unknown', 13),
 (22, 'M', 'Unknown', 2001, 2, 'Life Sciences', 'No', 'Unknown', 1),
 (44, 'M', 'Unknown', 1979, 4, 'Life Sciences', 'No', 'travel_rarely', 8),
 (35,
  'F',
  'Divorced',
  1988,
  4,
  'Life Sciences',
  'Yes',
  'travel_frequently',
  25),
 (25, 'M', 'Unknown', 1998, 1, 'Unknown', 'No', 'Unknown', 4),
 (24, 'M', 'Unknown', 1999, 1, 'Life Sciences', 'No', 'non-travel', 19),
 (38, 'M', 'Unknown', 1985, 3, 'Marketing', 'No', 'travel_rarely', 9),
 (29, 'F', 'Unknown', 1994, 3, 'Medical', 'Yes', 'travel_rarely', 1),
 (43, 'F', 'Married', 1980, 3, 'Life Sciences', 'No', 'travel_frequently', 1),
 (28, 'M', 'Single', 1995, 4, 'Medical', 'No', 'travel_rarely', 5),
 (20, 'F', 'Single', 2003, 3, 'Unknown', 'Yes', 'travel_rarely', 2),
 (47, 'F', 'Single', 1976, 4, 'Life Sciences', 'No', 'Unknown', 2),
 (28, 'F', 'Divorced', 1995, 3, 'Medical', 'No', 'Unknown', 9),
 (33, 'M', 'Single', 1990, 4, 'Unknown', 'No', 'travel_rarely', 1),
 (35, 'F', 'Unknown', 1988, 3, 'Unknown', 'No', 'travel_rarely', 48),
 (31, 'M', 'Single', 1992, 2, 'Medical', 'Yes', 'Unknown', 9),
 (33, 'F', 'Single', 1990, 1, 'Life Sciences', 'No', 'travel_rarely', 13),
 (27, 'F', 'Married', 1996, 4, 'Unknown', 'No', 'Unknown', 24),
 (35, 'M', 'Divorced', 1988, 4, 'Other', 'Yes', 'travel_rarely', 14),
 (30, 'F', 'Unknown', 1993, 3, 'Unknown', 'Yes', 'Unknown', 5),
 (48, 'M', 'Unknown', 1975, 3, 'Unknown', 'No', 'travel_rarely', 6),
 (27, 'F', 'Unknown', 1996, 4, 'Unknown', 'No', 'Unknown', 2),
 (35, 'M', 'Unknown', 1988, 3, 'Other', 'No', 'travel_rarely', 10),
 (26, 'M', 'Single', 1997, 2, 'Medical', 'Yes', 'non-travel', 29),
 (33, 'F', 'Unknown', 1990, 1, 'Life Sciences', 'No', 'non-travel', 8),
 (26, 'M', 'Married', 1997, 3, 'Life Sciences', 'Yes', 'Unknown', 2),
 (46, 'M', 'Unknown', 1977, 4, 'Medical', 'No', 'Unknown', 47),
 (20, 'F', 'Single', 2003, 3, 'Marketing', 'Yes', 'Unknown', 9),
 (30, 'M', 'Married', 1993, 3, 'Other', 'No', 'Unknown', 20),
 (27, 'M', 'Married', 1996, 5, 'Unknown', 'No', 'non-travel', 8),
 (55, 'M', 'Married', 1968, 1, 'Unknown', 'No', 'Unknown', 2),
 (33, 'M', 'Unknown', 1990, 3, 'Life Sciences', 'No', 'travel_rarely', 5),
 (28, 'M', 'Unknown', 1995, 2, 'Life Sciences', 'No', 'Unknown', 15),
 (34, 'M', 'Unknown', 1989, 1, 'Life Sciences', 'No', 'Unknown', 3),
 (33, 'M', 'Single', 1990, 3, 'Medical', 'Yes', 'Unknown', 25),
 (33, 'F', 'Married', 1990, 3, 'Life Sciences', 'No', 'Unknown', 7),
 (21, 'F', 'Single', 2002, 1, 'Unknown', 'No', 'Unknown', 1),
 (31, 'M', 'Married', 1992, 3, 'Unknown', 'No', 'travel_frequently', 20),
 (34, 'M', 'Single', 1989, 2, 'Unknown', 'No', 'Unknown', 2),
 (33, 'F', 'Single', 1990, 4, 'Unknown', 'Yes', 'Unknown', 9),
 (31, 'M', 'Unknown', 1992, 3, 'Medical', 'Yes', 'Unknown', 15),
 (37, 'F', 'Unknown', 1986, 2, 'Unknown', 'No', 'travel_rarely', 25),
 (27, 'M', 'Married', 1996, 3, 'Unknown', 'No', 'travel_rarely', 6),
 (35, 'M', 'Married', 1988, 4, 'Other', 'No', 'travel_frequently', 4),
 (45, 'F', 'Unknown', 1978, 3, 'Medical', 'No', 'Unknown', 7),
 (46, 'M', 'Unknown', 1977, 2, 'Life Sciences', 'No', 'Unknown', 15),
 (32, 'F', 'Married', 1991, 3, 'Other', 'No', 'Unknown', 13),
 (28, 'M', 'Single', 1995, 4, 'Medical', 'No', 'Unknown', 1),
 (35, 'M', 'Married', 1988, 1, 'Life Sciences', 'No', 'travel_rarely', 6),
 (28, 'M', 'Single', 1995, 3, 'Technical Degree', 'No', 'travel_rarely', 1),
 (25, 'F', 'Single', 1998, 3, 'Medical', 'No', 'Unknown', 1),
 (58, 'M', 'Unknown', 1965, 4, 'Life Sciences', 'No', 'Unknown', 23),
 (30, 'F', 'Married', 1993, 3, 'Technical Degree', 'No', 'travel_rarely', 6),
 (50, 'F', 'Married', 1973, 3, 'Life Sciences', 'No', 'Unknown', 46),
 (41, 'M', 'Single', 1982, 2, 'Medical', 'No', 'Unknown', 10),
 (31, 'F', 'Unknown', 1992, 3, 'Unknown', 'Yes', 'Unknown', 1),
 (44, 'M', 'Unknown', 1979, 3, 'Life Sciences', 'Yes', 'Unknown', 3),
 (25, 'M', 'Married', 1998, 1, 'Medical', 'No', 'travel_rarely', 19),
 (40, 'M', 'Unknown', 1983, 4, 'Life Sciences', 'No', 'travel_rarely', 1),
 (27, 'M', 'Single', 1996, 1, 'Life Sciences', 'No', 'travel_frequently', 2),
 (35, 'F', 'Married', 1988, 4, 'Marketing', 'Yes', 'Unknown', 18),
 (31, 'M', 'Married', 1992, 4, 'Unknown', 'Yes', 'Unknown', 22),
 (41, 'F', 'Unknown', 1982, 3, 'Unknown', 'No', 'Unknown', 22),
 (32, 'M', 'Married', 1991, 1, 'Marketing', 'No', 'Unknown', 2),
 (27, 'F', 'Married', 1996, 3, 'Marketing', 'No', 'Unknown', 10),
 (27, 'F', 'Single', 1996, 4, 'Life Sciences', 'Yes', 'travel_rarely', 22),
 (37, 'F', 'Single', 1986, 3, 'Medical', 'No', 'Unknown', 11),
 (35, 'M', 'Unknown', 1988, 2, 'Life Sciences', 'No', 'travel_frequently', 18),
 (25, 'M', 'Unknown', 1998, 3, 'Unknown', 'No', 'Unknown', 1),
 (30, 'F', 'Divorced', 1993, 4, 'Marketing', 'No', 'Unknown', 7),
 (36, 'F', 'Unknown', 1987, 4, 'Marketing', 'No', 'Unknown', 16),
 (25, 'M', 'Unknown', 1998, 2, 'Unknown', 'Yes', 'travel_rarely', 9),
 (56, 'F', 'Unknown', 1967, 2, 'Life Sciences', 'No', 'Unknown', 1),
 (41, 'M', 'Unknown', 1982, 5, 'Medical', 'No', 'Unknown', 5),
 (31, 'M', 'Unknown', 1992, 3, 'Life Sciences', 'Yes', 'travel_frequently', 2),
 (48, 'M', 'Divorced', 1975, 3, 'Marketing', 'No', 'Unknown', 7),
 (34, 'F', 'Married', 1989, 4, 'Unknown', 'Yes', 'Unknown', 9),
 (44, 'M', 'Single', 1979, 3, 'Life Sciences', 'No', 'Unknown', 5),
 (49, 'M', 'Married', 1974, 4, 'Unknown', 'No', 'travel_rarely', 5),
 (44, 'M', 'Unknown', 1979, 3, 'Life Sciences', 'No', 'Unknown', 20),
 (28, 'M', 'Married', 1995, 3, 'Unknown', 'Yes', 'Unknown', 1),
 (28, 'M', 'Single', 1995, 3, 'Life Sciences', 'No', 'travel_frequently', 7),
 (49, 'F', 'Married', 1974, 2, 'Medical', 'No', 'travel_rarely', 45),
 (29, 'F', 'Unknown', 1994, 1, 'Life Sciences', 'No', 'travel_frequently', 2),
 (32, 'M', 'Unknown', 1991, 3, 'Unknown', 'No', 'Unknown', 10),
 (31, 'F', 'Unknown', 1992, 3, 'Life Sciences', 'Yes', 'travel_frequently', 3),
 (40, 'M', 'Divorced', 1983, 2, 'Life Sciences', 'No', 'Unknown', 43),
 (33, 'M', 'Unknown', 1990, 4, 'Life Sciences', 'No', 'Unknown', 8),
 (30, 'M', 'Married', 1993, 4, 'Unknown', 'No', 'Unknown', 14),
 (38, 'M', 'Married', 1985, 3, 'Medical', 'Yes', 'travel_rarely', 2),
 (48, 'F', 'Married', 1975, 4, 'Unknown', 'No', 'travel_rarely', 15),
 (34, 'M', 'Unknown', 1989, 4, 'Unknown', 'No', 'Unknown', 1),
 (38, 'F', 'Single', 1985, 3, 'Life Sciences', 'No', 'travel_rarely', 16),
 (35, 'F', 'Single', 1988, 4, 'Unknown', 'No', 'travel_rarely', 10),
 (23, 'M', 'Married', 2000, 3, 'Unknown', 'No', 'Unknown', 10),
 (36, 'M', 'Unknown', 1987, 4, 'Unknown', 'No', 'Unknown', 6),
 (34, 'M', 'Unknown', 1989, 4, 'Unknown', 'No', 'travel_frequently', 36),
 (32, 'F', 'Married', 1991, 4, 'Unknown', 'No', 'Unknown', 5),
 (21, 'M', 'Single', 2002, 1, 'Medical', 'No', 'Unknown', 19),
 (40, 'F', 'Unknown', 1983, 3, 'Unknown', 'No', 'Unknown', 16),
 (48, 'M', 'Unknown', 1975, 5, 'Unknown', 'No', 'travel_frequently', 4),
 (35, 'M', 'Unknown', 1988, 2, 'Medical', 'Yes', 'Unknown', 10),
 (27, 'F', 'Unknown', 1996, 3, 'Medical', 'No', 'travel_rarely', 8),
 (27, 'M', 'Single', 1996, 3, 'Medical', 'No', 'Unknown', 2),
 (36, 'F', 'Married', 1987, 4, 'Life Sciences', 'No', 'Unknown', 5),
 (49, 'F', 'Married', 1974, 4, 'Other', 'No', 'travel_rarely', 22),
 (20, 'F', 'Single', 2003, 3, 'Medical', 'No', 'travel_rarely', 13),
 (36, 'F', 'Unknown', 1987, 4, 'Unknown', 'No', 'non-travel', 1),
 (33, 'M', 'Unknown', 1990, 4, 'Unknown', 'No', 'travel_rarely', 2),
 (49, 'F', 'Divorced', 1974, 3, 'Medical', 'No', 'Unknown', 16),
 (36, 'M', 'Married', 1987, 4, 'Unknown', 'No', 'travel_rarely', 26),
 (44, 'F', 'Unknown', 1979, 3, 'Unknown', 'No', 'travel_rarely', 1),
 (55, 'M', 'Divorced', 1968, 3, 'Technical Degree', 'No', 'Unknown', 12),
 (23, 'M', 'Divorced', 2000, 3, 'Life Sciences', 'Yes', 'Unknown', 7),
 (31, 'M', 'Married', 1992, 4, 'Life Sciences', 'No', 'Unknown', 2),
 (41, 'M', 'Married', 1982, 3, 'Human Resources', 'No', 'non-travel', 4),
 (36, 'M', 'Unknown', 1987, 5, 'Marketing', 'Yes', 'Unknown', 13),
 (23, 'F', 'Single', 2000, 1, 'Unknown', 'No', 'Unknown', 4),
 (24, 'M', 'Married', 1999, 3, 'Medical', 'No', 'Unknown', 35),
 (27, 'F', 'Unknown', 1996, 3, 'Unknown', 'No', 'Unknown', 48),
 (29, 'M', 'Unknown', 1994, 3, 'Unknown', 'No', 'Unknown', 1),
 (37, 'M', 'Unknown', 1986, 2, 'Marketing', 'No', 'travel_rarely', 9),
 (46, 'M', 'Unknown', 1977, 2, 'Life Sciences', 'No', 'Unknown', 26),
 (43, 'M', 'Unknown', 1980, 3, 'Unknown', 'No', 'non-travel', 9),
 (41, 'M', 'Single', 1982, 3, 'Unknown', 'No', 'travel_rarely', 35),
 (31, 'F', 'Unknown', 1992, 4, 'Unknown', 'No', 'Unknown', 7),
 (28, 'F', 'Unknown', 1995, 3, 'Medical', 'No', 'travel_rarely', 1),
 (38, 'F', 'Unknown', 1985, 4, 'Unknown', 'No', 'Unknown', 2),
 (58, 'M', 'Married', 1965, 4, 'Life Sciences', 'No', 'travel_frequently', 15),
 (29, 'M', 'Divorced', 1994, 3, 'Unknown', 'Yes', 'travel_rarely', 13),
 (47, 'F', 'Single', 1976, 4, 'Life Sciences', 'No', 'travel_rarely', 9),
 (28, 'M', 'Unknown', 1995, 2, 'Life Sciences', 'No', 'Unknown', 1),
 (55, 'M', 'Unknown', 1968, 4, 'Unknown', 'No', 'travel_rarely', 2),
 (29, 'M', 'Single', 1994, 3, 'Life Sciences', 'No', 'Unknown', 7),
 (35, 'F', 'Married', 1988, 2, 'Other', 'No', 'Unknown', 16),
 (34, 'M', 'Unknown', 1989, 4, 'Technical Degree', 'Yes', 'non-travel', 16),
 (56, 'M', 'Single', 1967, 2, 'Unknown', 'Yes', 'travel_rarely', 24),
 (36, 'M', 'Unknown', 1987, 2, 'Marketing', 'No', 'Unknown', 17),
 (45, 'M', 'Single', 1978, 4, 'Medical', 'No', 'travel_frequently', 1),
 (39, 'F', 'Unknown', 1984, 4, 'Unknown', 'No', 'Unknown', 21),
 (31, 'M', 'Divorced', 1992, 2, 'Unknown', 'No', 'Unknown', 3),
 (38, 'M', 'Unknown', 1985, 4, 'Other', 'No', 'travel_frequently', 1),
 (54, 'F', 'Single', 1969, 3, 'Unknown', 'No', 'Unknown', 10),
 (23, 'M', 'Single', 2000, 2, 'Unknown', 'No', 'Unknown', 12),
 (25, 'M', 'Unknown', 1998, 1, 'Other', 'No', 'Unknown', 2),
 (31, 'M', 'Unknown', 1992, 2, 'Life Sciences', 'No', 'Unknown', 2),
 (35, 'M', 'Married', 1988, 4, 'Medical', 'No', 'travel_rarely', 11),
 (22, 'M', 'Unknown', 2001, 2, 'Unknown', 'No', 'Unknown', 1),
 (32, 'F', 'Single', 1991, 4, 'Unknown', 'No', 'travel_rarely', 29),
 (29, 'M', 'Married', 1994, 4, 'Unknown', 'Yes', 'travel_rarely', 38),
 (32, 'M', 'Unknown', 1991, 3, 'Technical Degree', 'No', 'travel_rarely', 2),
 (40, 'M', 'Unknown', 1983, 4, 'Unknown', 'No', 'travel_rarely', 2),
 (50, 'M', 'Unknown', 1973, 3, 'Marketing', 'Yes', 'travel_rarely', 40),
 (26, 'F', 'Single', 1997, 3, 'Other', 'No', 'Unknown', 5),
 (36, 'M', 'Unknown', 1987, 2, 'Medical', 'No', 'travel_frequently', 23),
 (27, 'M', 'Unknown', 1996, 3, 'Life Sciences', 'No', 'travel_rarely', 4),
 (33, 'M', 'Unknown', 1990, 3, 'Medical', 'No', 'Unknown', 2),
 (36, 'F', 'Married', 1987, 4, 'Life Sciences', 'No', 'Unknown', 5),
 (31, 'M', 'Single', 1992, 4, 'Medical', 'No', 'Unknown', 1),
 (22, 'M', 'Unknown', 2001, 1, 'Technical Degree', 'No', 'travel_rarely', 2),
 (27, 'M', 'Divorced', 1996, 3, 'Unknown', 'No', 'Unknown', 3),
 (33, 'M', 'Unknown', 1990, 3, 'Life Sciences', 'No', 'travel_rarely', 2),
 (36, 'F', 'Single', 1987, 1, 'Unknown', 'No', 'travel_frequently', 18),
 (37, 'F', 'Unknown', 1986, 2, 'Medical', 'No', 'Unknown', 18),
 (49, 'F', 'Unknown', 1974, 2, 'Technical Degree', 'No', 'Unknown', 45),
 (35, 'M', 'Single', 1988, 4, 'Unknown', 'No', 'Unknown', 9),
 (22, 'F', 'Married', 2001, 3, 'Medical', 'No', 'Unknown', 11),
 (33, 'F', 'Unknown', 1990, 5, 'Unknown', 'No', 'Unknown', 1),
 (31, 'M', 'Divorced', 1992, 4, 'Medical', 'Yes', 'Unknown', 13),
 (45, 'F', 'Unknown', 1978, 4, 'Unknown', 'Yes', 'Unknown', 26),
 (53, 'M', 'Married', 1970, 3, 'Unknown', 'No', 'Unknown', 7),
 (34, 'F', 'Unknown', 1989, 2, 'Unknown', 'No', 'travel_rarely', 8),
 (28, 'M', 'Divorced', 1995, 2, 'Medical', 'No', 'travel_rarely', 8),
 (42, 'M', 'Single', 1981, 1, 'Unknown', 'No', 'travel_frequently', 2),
 (40, 'M', 'Unknown', 1983, 4, 'Unknown', 'No', 'Unknown', 9),
 (37, 'M', 'Married', 1986, 4, 'Life Sciences', 'Yes', 'Unknown', 1),
 (38, 'M', 'Married', 1985, 1, 'Unknown', 'No', 'Unknown', 10),
 (34, 'F', 'Married', 1989, 4, 'Life Sciences', 'No', 'travel_rarely', 10),
 (50, 'M', 'Married', 1973, 4, 'Medical', 'No', 'travel_rarely', 12),
 (26, 'M', 'Unknown', 1997, 3, 'Unknown', 'Yes', 'Unknown', 21),
 (40, 'F', 'Married', 1983, 2, 'Medical', 'No', 'Unknown', 29),
 (19, 'M', 'Single', 2004, 2, 'Unknown', 'No', 'travel_rarely', 47),
 (20, 'M', 'Single', 2003, 3, 'Marketing', 'No', 'travel_rarely', 21),
 (59, 'F', 'Single', 1964, 3, 'Unknown', 'No', 'Unknown', 3),
 (30, 'F', 'Unknown', 1993, 4, 'Medical', 'No', 'travel_rarely', 17),
 (45, 'M', 'Married', 1978, 3, 'Unknown', 'No', 'travel_rarely', 29),
 (30, 'M', 'Unknown', 1993, 1, 'Life Sciences', 'No', 'Unknown', 42),
 (24, 'M', 'Unknown', 1999, 2, 'Technical Degree', 'No', 'Unknown', 24),
 (32, 'F', 'Married', 1991, 2, 'Life Sciences', 'No', 'Unknown', 5),
 (28, 'M', 'Married', 1995, 3, 'Medical', 'Yes', 'travel_rarely', 24),
 (49, 'F', 'Single', 1974, 4, 'Unknown', 'No', 'Unknown', 11),
 (38, 'F', 'Married', 1985, 2, 'Medical', 'No', 'travel_frequently', 2),
 (46, 'M', 'Unknown', 1977, 4, 'Unknown', 'No', 'travel_rarely', 2),
 (34, 'M', 'Unknown', 1989, 4, 'Life Sciences', 'Yes', 'travel_frequently', 9),
 (39, 'M', 'Unknown', 1984, 3, 'Life Sciences', 'No', 'Unknown', 20),
 (26, 'F', 'Married', 1997, 2, 'Unknown', 'Yes', 'travel_rarely', 20),
 (54, 'F', 'Married', 1969, 4, 'Medical', 'No', 'travel_rarely', 7),
 (36, 'M', 'Unknown', 1987, 1, 'Unknown', 'No', 'travel_frequently', 2),
 (32, 'F', 'Unknown', 1991, 2, 'Unknown', 'Yes', 'travel_frequently', 5),
 (33, 'M', 'Divorced', 1990, 2, 'Medical', 'No', 'Unknown', 1),
 (37, 'M', 'Single', 1986, 2, 'Unknown', 'Yes', 'travel_rarely', 2),
 (59, 'F', 'Unknown', 1964, 3, 'Medical', 'No', 'travel_rarely', 3),
 (21, 'M', 'Single', 2002, 2, 'Unknown', 'No', 'travel_rarely', 15),
 (30, 'M', 'Unknown', 1993, 2, 'Medical', 'No', 'travel_rarely', 9),
 (39, 'M', 'Married', 1984, 3, 'Technical Degree', 'Yes', 'travel_rarely', 18),
 (35, 'M', 'Divorced', 1988, 2, 'Other', 'No', 'Unknown', 4),
 (27, 'F', 'Divorced', 1996, 4, 'Life Sciences', 'No', 'Unknown', 2),
 (26, 'M', 'Single', 1997, 3, 'Life Sciences', 'Yes', 'Unknown', 25),
 (48, 'M', 'Unknown', 1975, 2, 'Unknown', 'Yes', 'Unknown', 1),
 (35, 'M', 'Married', 1988, 2, 'Medical', 'No', 'non-travel', 11),
 (55, 'F', 'Divorced', 1968, 3, 'Unknown', 'No', 'travel_rarely', 8),
 (36, 'F', 'Unknown', 1987, 3, 'Life Sciences', 'No', 'travel_rarely', 6),
 (55, 'M', 'Married', 1968, 2, 'Unknown', 'No', 'Unknown', 1),
 (23, 'M', 'Divorced', 2000, 1, 'Technical Degree', 'No', 'Unknown', 2),
 (20, 'F', 'Single', 2003, 3, 'Life Sciences', 'Yes', 'Unknown', 6),
 (37, 'M', 'Divorced', 1986, 2, 'Life Sciences', 'No', 'Unknown', 2),
 (22, 'F', 'Single', 2001, 3, 'Unknown', 'No', 'Unknown', 15),
 (36, 'F', 'Divorced', 1987, 2, 'Life Sciences', 'No', 'Unknown', 3),
 (19, 'M', 'Single', 2004, 1, 'Marketing', 'Yes', 'travel_rarely', 22),
 (41, 'M', 'Unknown', 1982, 4, 'Life Sciences', 'No', 'Unknown', 9),
 (19, 'F', 'Single', 2004, 1, 'Unknown', 'No', 'Unknown', 18),
 (51, 'M', 'Married', 1972, 4, 'Unknown', 'No', 'Unknown', 7),
 (58, 'F', 'Married', 1965, 3, 'Unknown', 'No', 'Unknown', 9),
 (19, 'M', 'Single', 2004, 3, 'Life Sciences', 'Yes', 'travel_rarely', 2),
 (38, 'M', 'Unknown', 1985, 4, 'Unknown', 'No', 'travel_rarely', 38),
 (38, 'M', 'Married', 1985, 1, 'Medical', 'Yes', 'Unknown', 29),
 (59, 'M', 'Married', 1964, 3, 'Life Sciences', 'No', 'travel_rarely', 3),
 (50, 'F', 'Divorced', 1973, 4, 'Medical', 'No', 'travel_rarely', 1),
 (41, 'M', 'Divorced', 1982, 2, 'Life Sciences', 'No', 'travel_rarely', 19),
 (37, 'M', 'Unknown', 1986, 3, 'Medical', 'Yes', 'Unknown', 10),
 (28, 'M', 'Single', 1995, 4, 'Life Sciences', 'Yes', 'travel_rarely', 2),
 (31, 'M', 'Unknown', 1992, 3, 'Medical', 'No', 'travel_rarely', 23),
 (44,
  'M',
  'Unknown',
  1979,
  3,
  'Life Sciences',
  'Yes',
  'travel_frequently',
  12),
 (36, 'M', 'Single', 1987, 3, 'Technical Degree', 'No', 'Unknown', 3),
 (36, 'M', 'Married', 1987, 4, 'Life Sciences', 'No', 'Unknown', 18),
 (39, 'F', 'Married', 1984, 2, 'Unknown', 'Yes', 'travel_rarely', 3),
 (42, 'F', 'Single', 1981, 5, 'Unknown', 'No', 'travel_rarely', 23),
 (32, 'F', 'Married', 1991, 3, 'Unknown', 'No', 'Unknown', 2),
 (48, 'F', 'Married', 1975, 1, 'Medical', 'No', 'travel_rarely', 29),
 (42, 'M', 'Unknown', 1981, 4, 'Unknown', 'No', 'travel_rarely', 2),
 (36, 'F', 'Unknown', 1987, 5, 'Medical', 'No', 'non-travel', 1),
 (33, 'F', 'Single', 1990, 3, 'Marketing', 'Yes', 'travel_rarely', 5),
 (46, 'F', 'Married', 1977, 4, 'Medical', 'No', 'Unknown', 7),
 (21, 'F', 'Single', 2002, 3, 'Life Sciences', 'Yes', 'travel_rarely', 12),
 (30, 'M', 'Unknown', 1993, 1, 'Technical Degree', 'No', 'Unknown', 2),
 (30, 'M', 'Single', 1993, 3, 'Unknown', 'Yes', 'travel_frequently', 4),
 (32, 'M', 'Single', 1986, 4, 'Unknown', 'No', 'travel_rarely', 10),
 (38, 'M', 'Married', 1985, 2, 'Unknown', 'No', 'travel_frequently', 2),
 (19, 'M', 'Single', 2004, 2, 'Technical Degree', 'Yes', 'Unknown', 2),
 (57, 'M', 'Divorced', 1966, 3, 'Unknown', 'No', 'travel_rarely', 29),
 (18, 'M', 'Unknown', 2005, 3, 'Marketing', 'Yes', 'travel_frequently', 5),
 (40, 'M', 'Unknown', 1983, 3, 'Other', 'No', 'Unknown', 28),
 (26, 'M', 'Single', 1997, 3, 'Unknown', 'Yes', 'travel_rarely', 24),
 (24, 'M', 'Married', 1999, 2, 'Other', 'No', 'Unknown', 17),
 (30, 'M', 'Married', 1993, 4, 'Unknown', 'Yes', 'travel_frequently', 12),
 (37, 'M', 'Married', 1986, 3, 'Marketing', 'No', 'travel_rarely', 2),
 (20, 'F', 'Unknown', 2003, 3, 'Life Sciences', 'No', 'travel_rarely', 1),
 (43, 'M', 'Divorced', 1980, 4, 'Other', 'No', 'Unknown', 14),
 (38, 'F', 'Unknown', 1985, 1, 'Life Sciences', 'No', 'Unknown', 27),
 (29, 'F', 'Unknown', 1994, 2, 'Medical', 'No', 'Unknown', 44),
 (20, 'M', 'Single', 2003, 1, 'Medical', 'Yes', 'Unknown', 10),
 (35, 'M', 'Married', 1988, 3, 'Unknown', 'No', 'non-travel', 20),
 (37, 'M', 'Single', 1986, 2, 'Life Sciences', 'No', 'Unknown', 10),
 (38, 'F', 'Single', 1985, 3, 'Unknown', 'No', 'Unknown', 1),
 (29, 'M', 'Single', 1994, 3, 'Unknown', 'No', 'Unknown', 10),
 (38, 'M', 'Married', 1985, 5, 'Medical', 'No', 'travel_rarely', 2),
 (34, 'F', 'Unknown', 1989, 5, 'Life Sciences', 'No', 'travel_rarely', 8),
 (23, 'M', 'Unknown', 2000, 3, 'Life Sciences', 'Yes', 'Unknown', 6),
 (24, 'M', 'Divorced', 1999, 3, 'Life Sciences', 'No', 'Unknown', 9),
 (29, 'F', 'Married', 1994, 2, 'Life Sciences', 'Yes', 'travel_rarely', 1),
 (40, 'M', 'Unknown', 1983, 4, 'Medical', 'No', 'travel_frequently', 16),
 (30, 'F', 'Single', 1993, 3, 'Unknown', 'No', 'travel_rarely', 2),
 (45, 'F', 'Unknown', 1978, 3, 'Unknown', 'No', 'Unknown', 2),
 (53, 'M', 'Married', 1970, 3, 'Marketing', 'No', 'travel_rarely', 8),
 (37, 'M', 'Single', 1986, 2, 'Unknown', 'No', 'Unknown', 19),
 (32, 'M', 'Unknown', 1991, 4, 'Unknown', 'Yes', 'travel_rarely', 25),
 (58, 'M', 'Divorced', 1965, 1, 'Unknown', 'Yes', 'travel_frequently', 2),
 (21, 'F', 'Single', 2002, 1, 'Other', 'Yes', 'travel_rarely', 18),
 (34, 'M', 'Unknown', 1989, 3, 'Life Sciences', 'No', 'Unknown', 48),
 (58, 'F', 'Divorced', 1965, 3, 'Unknown', 'No', 'Unknown', 5),
 (48, 'M', 'Married', 1975, 4, 'Medical', 'No', 'Unknown', 20),
 (32, 'F', 'Married', 1991, 3, 'Life Sciences', 'No', 'non-travel', 1),
 (25, 'M', 'Married', 1998, 2, 'Marketing', 'Yes', 'Unknown', 19),
 (40, 'M', 'Divorced', 1983, 4, 'Marketing', 'No', 'Unknown', 10),
 (19, 'M', 'Single', 2004, 3, 'Other', 'Yes', 'Unknown', 21),
 (40, 'F', 'Single', 1983, 3, 'Unknown', 'Yes', 'non-travel', 24),
 (29, 'F', 'Single', 1994, 3, 'Unknown', 'Yes', 'travel_rarely', 10),
 (47, 'F', 'Single', 1976, 2, 'Life Sciences', 'No', 'travel_rarely', 18),
 (18, 'M', 'Unknown', 2005, 2, 'Unknown', 'No', 'Unknown', 5),
 (42, 'M', 'Unknown', 1981, 2, 'Unknown', 'No', 'Unknown', 5),
 (37, 'F', 'Unknown', 1986, 2, 'Medical', 'Yes', 'travel_rarely', 11),
 (36, 'M', 'Divorced', 1987, 3, 'Unknown', 'Yes', 'Unknown', 15),
 (28, 'M', 'Married', 1995, 1, 'Medical', 'No', 'travel_rarely', 10),
 (43, 'M', 'Unknown', 1980, 3, 'Unknown', 'No', 'Unknown', 25),
 (21, 'F', 'Single', 2002, 3, 'Unknown', 'Yes', 'travel_rarely', 10),
]

try: 
    mycursor.executemany(sql, val)
    cnx.commit()
    print(mycursor.rowcount, "registro/s insertado/s.")

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

Failed executing the operation; Not all parameters were used in the SQL statement
Error Code: -1
SQLSTATE None
Message Failed executing the operation; Not all parameters were used in the SQL statement
